https://stackoverflow.com/questions/56854243/how-to-link-the-libraries-when-executing-cuda-program-on-google-colab

In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-s8c6ynei
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-s8c6ynei
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=f41b80727f3eb237bbed1bdecb131c2623c100fdb5da4a9aed9bc634ae3e9769
  Stored in directory: /tmp/pip-ephem-wheel-cache-km8vrxzf/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [3]:
%%cuda --name kernel.cu 
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <assert.h>

#include <cuda_runtime.h>
#include <cusolverDn.h>

#include "Utilities.h"
#include "TimingGPU.h"

//#define FULLSVD
//#define PRINTRESULTS

/********/
/* MAIN */
/********/
int main() {

    const int           M           = 3;
    const int           N           = 3;
    const int           lda         = M;
    //const int         numMatrices = 3;
    const int           numMatrices = 16384;

    TimingGPU timerGPU;

    // --- Setting the host matrix
    double *h_A = (double *)malloc(lda * N * numMatrices * sizeof(double));
    for (unsigned int k = 0; k < numMatrices; k++)
        for (unsigned int i = 0; i < M; i++){
            for (unsigned int j = 0; j < N; j++){
                h_A[k * M * N + j * M + i] = (1. / (k + 1)) * (i + j * j) * (i + j);
                //printf("%d %d %f\n", i, j, h_A[j*M + i]);
            }
        }

    // --- Setting the device matrix and moving the host matrix to the device
    double *d_A;         gpuErrchk(cudaMalloc(&d_A, M * N * numMatrices * sizeof(double)));
    gpuErrchk(cudaMemcpy(d_A, h_A, M * N * numMatrices * sizeof(double), cudaMemcpyHostToDevice));

    // --- host side SVD results space
    double *h_S = (double *)malloc(N *     numMatrices * sizeof(double));
    double *h_U = NULL;
    double *h_V = NULL;
#ifdef FULLSVD
            h_U = (double *)malloc(M * M * numMatrices * sizeof(double));
            h_V = (double *)malloc(N * N * numMatrices * sizeof(double));
#endif

    // --- device side SVD workspace and matrices
    int work_size = 0;

    int *devInfo;        gpuErrchk(cudaMalloc(&devInfo, sizeof(int)));
    double *d_S;         gpuErrchk(cudaMalloc(&d_S, N *     numMatrices * sizeof(double)));
    double *d_U = NULL;
    double *d_V = NULL;
#ifdef FULLSVD
                         gpuErrchk(cudaMalloc(&d_U, M * M * numMatrices * sizeof(double)));
                         gpuErrchk(cudaMalloc(&d_V, N * N * numMatrices * sizeof(double)));
#endif

    double *d_work = NULL; /* devie workspace for gesvdj */
    int devInfo_h = 0; /* host copy of error devInfo_h */

    // --- Parameters configuration of Jacobi-based SVD
    const double            tol             = 1.e-7;
    const int               maxSweeps       = 15;
          cusolverEigMode_t jobz;                                   // --- CUSOLVER_EIG_MODE_VECTOR - Compute eigenvectors; CUSOLVER_EIG_MODE_NOVECTOR - Compute singular values only
#ifdef FULLSVD
        jobz = CUSOLVER_EIG_MODE_VECTOR;
#else
        jobz = CUSOLVER_EIG_MODE_NOVECTOR;
#endif

    const int               econ            = 0;                            // --- econ = 1 for economy size 

    // --- Numerical result parameters of gesvdj 
    double                  residual        = 0;
    int                     executedSweeps  = 0;

    // --- CUDA solver initialization
    cusolverDnHandle_t solver_handle = NULL;
    cusolveSafeCall(cusolverDnCreate(&solver_handle));

    // --- Configuration of gesvdj
    gesvdjInfo_t gesvdj_params = NULL;
    cusolveSafeCall(cusolverDnCreateGesvdjInfo(&gesvdj_params));

    // --- Set the computation tolerance, since the default tolerance is machine precision
    cusolveSafeCall(cusolverDnXgesvdjSetTolerance(gesvdj_params, tol));

    // --- Set the maximum number of sweeps, since the default value of max. sweeps is 100
    cusolveSafeCall(cusolverDnXgesvdjSetMaxSweeps(gesvdj_params, maxSweeps));

    // --- Query the SVD workspace 
    cusolveSafeCall(cusolverDnDgesvdjBatched_bufferSize(
        solver_handle,
        jobz,                                       // --- Compute the singular vectors or not
        M,                                          // --- Nubmer of rows of A, 0 <= M
        N,                                          // --- Number of columns of A, 0 <= N 
        d_A,                                        // --- M x N
        lda,                                        // --- Leading dimension of A
        d_S,                                        // --- Square matrix of size min(M, N) x min(M, N)
        d_U,                                        // --- M x M if econ = 0, M x min(M, N) if econ = 1
        lda,                                        // --- Leading dimension of U, ldu >= max(1, M)
        d_V,                                        // --- N x N if econ = 0, N x min(M,N) if econ = 1
        lda,                                        // --- Leading dimension of V, ldv >= max(1, N)
        &work_size,
        gesvdj_params,
        numMatrices));

    gpuErrchk(cudaMalloc(&d_work, sizeof(double) * work_size));

    // --- Compute SVD
    timerGPU.StartCounter();
    cusolveSafeCall(cusolverDnDgesvdjBatched(
        solver_handle,
        jobz,                                       // --- Compute the singular vectors or not
        M,                                          // --- Number of rows of A, 0 <= M
        N,                                          // --- Number of columns of A, 0 <= N 
        d_A,                                        // --- M x N
        lda,                                        // --- Leading dimension of A
        d_S,                                        // --- Square matrix of size min(M, N) x min(M, N)
        d_U,                                        // --- M x M if econ = 0, M x min(M, N) if econ = 1
        lda,                                        // --- Leading dimension of U, ldu >= max(1, M)
        d_V,                                        // --- N x N if econ = 0, N x min(M, N) if econ = 1
        lda,                                        // --- Leading dimension of V, ldv >= max(1, N)
        d_work,
        work_size,
        devInfo,
        gesvdj_params,
        numMatrices));

    printf("Calculation of the singular values only: %f ms\n\n", timerGPU.GetCounter());

    gpuErrchk(cudaMemcpy(&devInfo_h, devInfo, sizeof(int), cudaMemcpyDeviceToHost));
    gpuErrchk(cudaMemcpy(h_S, d_S, sizeof(double) *       N * numMatrices, cudaMemcpyDeviceToHost));
#ifdef FULLSVD
    gpuErrchk(cudaMemcpy(h_U, d_U, sizeof(double) * lda * M * numMatrices, cudaMemcpyDeviceToHost));
    gpuErrchk(cudaMemcpy(h_V, d_V, sizeof(double) * lda * N * numMatrices, cudaMemcpyDeviceToHost));
#endif

#ifdef PRINTRESULTS
    printf("SINGULAR VALUES \n");
    printf("_______________ \n");
    for (int k = 0; k < numMatrices; k++) {
        for (int p = 0; p < N; p++)
            printf("Matrix nr. %d; SV nr. %d; Value = %f\n", k, p, h_S[k * N + p]);
        printf("\n");
    }
#ifdef FULLSVD
    printf("SINGULAR VECTORS U \n");
    printf("__________________ \n");
    for (int k = 0; k < numMatrices; k++) {
        for (int q = 0; q < (1 - econ) * M + econ * min(M, N); q++)
            for (int p = 0; p < M; p++)
                printf("Matrix nr. %d; U nr. %d; Value = %f\n", k, p, h_U[((1 - econ) * M + econ * min(M, N)) * M * k + q * M + p]);
        printf("\n");
    }

    printf("SINGULAR VECTORS V \n");
    printf("__________________ \n");
    for (int k = 0; k < numMatrices; k++) {
        for (int q = 0; q < (1 - econ) * N + econ * min(M, N); q++)
            for (int p = 0; p < N; p++)
                printf("Matrix nr. %d; V nr. %d; Value = %f\n", k, p, h_V[((1 - econ) * N + econ * min(M, N)) * N * k + q * N + p]);
        printf("\n");
    }
#endif
#endif

    if (0 == devInfo_h){
        printf("gesvdj converges \n");
    }
    else if (0 > devInfo_h){
        printf("%d-th parameter is wrong \n", -devInfo_h);
        exit(1);
    }
    else{
        printf("WARNING: devInfo_h = %d : gesvdj does not converge \n", devInfo_h);
    }

    // --- Free resources
    if (d_A) gpuErrchk(cudaFree(d_A));
    if (d_S) gpuErrchk(cudaFree(d_S));
#ifdef FULLSVD
    if (d_U) gpuErrchk(cudaFree(d_U));
    if (d_V) gpuErrchk(cudaFree(d_V));
#endif
    if (devInfo) gpuErrchk(cudaFree(devInfo));
    if (d_work) gpuErrchk(cudaFree(d_work));
    if (solver_handle) cusolveSafeCall(cusolverDnDestroy(solver_handle));
    if (gesvdj_params) cusolveSafeCall(cusolverDnDestroyGesvdjInfo(gesvdj_params));

    gpuErrchk(cudaDeviceReset());

    return 0;
}

'File written in /content/src/kernel.cu'

In [4]:
%%cuda --name InputOutput.cu 
#include <fstream>
#include <iomanip>

#include <cuda.h>

#include "InputOutput.h"
#include "BBComplex.h"
#include "Utilities.h"

#define prec_save 10

/***********************************************/
/* SAVE INDIVIDUAL REAL GPU MATRIX TO txt FILE */
/***********************************************/
template <class T>
void saveGPUrealtxt(const T * d_in, const char *filename, const int M) {
	
	T *h_in	= (T *)malloc(M * sizeof(T));
	
	gpuErrchk(cudaMemcpy(h_in, d_in, M * sizeof(T), cudaMemcpyDeviceToHost));
	
	std::ofstream outfile;
	outfile.open(filename);
	for(int i = 0; i < M; i++) outfile << std::setprecision(prec_save) << h_in[i] << "\n"; 
	outfile.close();

}

template void saveGPUrealtxt<float> (const float  *, const char *, const int);
template void saveGPUrealtxt<double>(const double *, const char *, const int);

/***********************************************/
/* SAVE INDIVIDUAL REAL CPU MATRIX TO txt FILE */
/***********************************************/
template <class T>
void saveCPUrealtxt(const T * h_in, const char *filename, const int M) {
	
	std::ofstream outfile;
	outfile.open(filename);
	for(int i = 0; i < M; i++) outfile << std::setprecision(prec_save) << h_in[i] << "\n"; 
	outfile.close();

}

template void saveCPUrealtxt<float> (const float  *, const char *, const int);
template void saveCPUrealtxt<double>(const double *, const char *, const int);

/**************************************************/
/* SAVE INDIVIDUAL COMPLEX GPU MATRIX TO txt FILE */
/**************************************************/
template <class T>
void saveGPUcomplextxt(const T * d_in, const char *filename, const int M) {
	
	T *h_in	= (T *)malloc(M * sizeof(T));
	
	gpuErrchk(cudaMemcpy(h_in, d_in, M * sizeof(T), cudaMemcpyDeviceToHost));
	
	std::ofstream outfile;
	outfile.open(filename);
	for(int i = 0; i < M; i++) { 
		//printf("%f %f\n", h_in[i].c.x, h_in[i].c.y);
		outfile << std::setprecision(prec_save) << h_in[i].c.x << "\n"; outfile << std::setprecision(prec_save) << h_in[i].c.y << "\n";
	}
	outfile.close();

}

void saveGPUcomplextxt(const float2 * d_in, const char *filename, const int M) {
	
	float2 *h_in	= (float2 *)malloc(M * sizeof(float2));
	
	gpuErrchk(cudaMemcpy(h_in, d_in, M * sizeof(float2), cudaMemcpyDeviceToHost));
	
	std::ofstream outfile;
	outfile.open(filename);
	for(int i = 0; i < M; i++) { 
		//printf("%f %f\n", h_in[i].c.x, h_in[i].c.y);
		outfile << std::setprecision(prec_save) << h_in[i].x << "\n"; outfile << std::setprecision(prec_save) << h_in[i].y << "\n";
	}
	outfile.close();

}

void saveGPUcomplextxt(const double2 * d_in, const char *filename, const int M) {
	
	double2 *h_in	= (double2 *)malloc(M * sizeof(double2));
	
	gpuErrchk(cudaMemcpy(h_in, d_in, M * sizeof(double2), cudaMemcpyDeviceToHost));
	
	std::ofstream outfile;
	outfile.open(filename);
	for(int i = 0; i < M; i++) { 
		//printf("%f %f\n", h_in[i].c.x, h_in[i].c.y);
		outfile << std::setprecision(prec_save) << h_in[i].x << "\n"; outfile << std::setprecision(prec_save) << h_in[i].y << "\n";
	}
	outfile.close();

}

template void saveGPUcomplextxt<float2_> (const float2_  *, const char *, const int);
template void saveGPUcomplextxt<double2_>(const double2_ *, const char *, const int);

/****************************************************/
/* LOAD INDIVIDUAL REAL MATRIX FROM txt FILE TO CPU */
/****************************************************/
// --- Load individual real matrix from txt file
template <class T>
T * loadCPUrealtxt(const char *filename, T * __restrict__ h_out, const int M) {
		
	h_out	= (T *)malloc(M * sizeof(T));

	std::ifstream infile;
	infile.open(filename);
	for(int i = 0; i < M; i++) infile >> h_out[i]; 
	infile.close();

	return h_out;

}

template float  * loadCPUrealtxt<float> (const char *, float  * __restrict__, const int);
template double * loadCPUrealtxt<double>(const char *, double * __restrict__, const int);

/****************************************************/
/* LOAD INDIVIDUAL REAL MATRIX FROM txt FILE TO GPU */
/****************************************************/
// --- Load individual real matrix from txt file
template <class T>
T * loadGPUrealtxt(const char *filename, T * __restrict__ d_out, const int M) {
		
	T *h_out	= (T *)malloc(M * sizeof(T));
	//T *d_out;	gpuErrchk(cudaMalloc(&d_out, M * sizeof(T)));
	gpuErrchk(cudaMalloc((void**)&d_out, M * sizeof(T)));

	std::ifstream infile;
	infile.open(filename);
	for(int i = 0; i < M; i++) infile >> h_out[i]; 
	infile.close();

	gpuErrchk(cudaMemcpy(d_out, h_out, M * sizeof(T), cudaMemcpyHostToDevice));
	
	return d_out;

}

template float  * loadGPUrealtxt<float> (const char *, float  * __restrict__, const int);
template double * loadGPUrealtxt<double>(const char *, double * __restrict__, const int);



'File written in /content/src/InputOutput.cu'

In [5]:
%%cuda --name InputOutput.h
#ifndef INPUTOUTPUT_CUH
#define INPUTOUTPUT_CUH

template <class T>
void saveGPUrealtxt(const T *, const char *, const int);

template <class T>
void saveCPUrealtxt(const T *, const char *, const int);

template <class T>
void saveGPUcomplextxt(const T *, const char *, const int);

void saveGPUcomplextxt(const double2 *, const char *, const int);

template <class T>
T * loadCPUrealtxt(const char *, T * __restrict__, const int);

template <class T>
T * loadGPUrealtxt(const char *, T * __restrict__, const int);

#endif

'File written in /content/src/InputOutput.h'

In [6]:
%%cuda --name step1.h
#ifndef __STEP1_H__
#define __STEP1_H__

//#define DEBUG_REARRANGE
//#define DEBUG_BIDIAGONALIZE
//#define DEBUG_EXTRACT_DIAGONALS

/*************************************/
/* EXTRACT DIAGONALS KERNEL FUNCTION */
/*************************************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Nrows, const unsigned int Ncols>
__global__ void extractDiagonals(const T * __restrict__ inputMatrices, T * __restrict__ d, T * __restrict__ e) {
#else
template<class T>
__global__ void extractDiagonals(const T * __restrict__ inputMatrices, T * __restrict__ d, T * __restrict__ e,
	                             const unsigned int numMatrices, const unsigned int Nrows, const unsigned int Ncols) {
#endif

	int tid = threadIdx.x + blockDim.x * blockIdx.x;

	if (tid < numMatrices) {

		//for (unsigned int i = 0; i < Ncols; i++)     d[tid + i * numMatrices] = inputMatrices[tid + i * numMatrices * (Ncols + 1)];
		//for (unsigned int i = 0; i < Ncols - 1; i++) e[tid + i * numMatrices] = inputMatrices[tid + numMatrices + i * numMatrices * (Ncols + 1)];
#pragma unroll
		for (unsigned int i = 0; i < Ncols - 1; i++) {
			e[tid + i * numMatrices] = inputMatrices[tid + numMatrices + i * numMatrices * (Ncols + 1)];
			d[tid + i * numMatrices] = inputMatrices[tid + i * numMatrices * (Ncols + 1)];
		}
		d[tid + (Ncols - 1) * numMatrices] = inputMatrices[tid + (Ncols - 1) * numMatrices * (Ncols + 1)];

	}
}

/************************************/
/* COMPUTE LEFT HOUSEHOLDER VECTORS */
/************************************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Nrows, const unsigned int Ncols>
__device__ __forceinline__ void computeLeftHouseholderVectors(const T * __restrict__ inputMatrices, T * __restrict__ outputLeftHourseholderVectors, T &betaq,
											  const unsigned int tid, const unsigned int iterCounter) {
#else
template<class T>
__device__ __forceinline__ void computeLeftHouseholderVectors(const T * __restrict__ inputMatrices, T * __restrict__ outputLeftHourseholderVectors, T &betaq,
											  const unsigned int tid, const unsigned int iterCounter, const unsigned int numMatrices,
											  const unsigned int Nrows, const unsigned int Ncols) {
#endif

	T x0, norm2 = static_cast<T>(0), mu, v0;
#pragma unroll
	for (unsigned int i = 0; i < iterCounter ; i++) outputLeftHourseholderVectors[i] = static_cast<T>(0);

#pragma unroll
	for (unsigned int i = iterCounter + 1; i < Nrows; i++) {
		const T buffer = inputMatrices[tid + iterCounter * numMatrices + i * numMatrices * Ncols];
		outputLeftHourseholderVectors[i] = buffer;
        // --- Evaluate the squared norm of input vector x
        norm2 = norm2 + buffer * buffer;
	}

	x0 = inputMatrices[tid + (iterCounter * numMatrices) * (1 + Ncols)];

	if (norm2 == static_cast<T>(0)) betaq = static_cast<T>(0);
    else {
		mu = sqrt(x0 * x0 + norm2);

		if (x0 <= 0)	v0 = x0 - mu;
        else			v0 = -norm2 / (x0 + mu);

        T temp = static_cast<T>(1) / v0;
        betaq = static_cast<T>(2) * ((v0 * v0) / (norm2 + (v0 * v0)));
        outputLeftHourseholderVectors[iterCounter] = static_cast<T>(1);
#pragma unroll
		for(unsigned int s = iterCounter + 1; s < Nrows ; s++) outputLeftHourseholderVectors[s] *= temp;
	}
}

/*************************************/
/* COMPUTE RIGHT HOUSEHOLDER VECTORS */
/*************************************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Nrows, const unsigned int Ncols>
__device__ __forceinline__ void computeRightHouseholderVectors(const T * __restrict__ inputVectors, T * __restrict__ outputRightHouseholderVectors, T &betap, const unsigned int tid,
											   const unsigned int iterCounter) {
#else
template<class T>
__device__ __forceinline__ void computeRightHouseholderVectors(const T * __restrict__ inputVectors, T * __restrict__ outputRightHouseholderVectors, T &betap, const unsigned int tid,
											   const unsigned int iterCounter, const unsigned int numMatrices, const unsigned int Nrows,
											   const unsigned int Ncols) {
#endif

	T x0, norm2 = static_cast<T>(0), mu, v0;
#pragma unroll
	for (unsigned int i = 0; i < iterCounter + 1 ; i++) outputRightHouseholderVectors[i] = static_cast<T>(0);

#pragma unroll
	for (unsigned int i = iterCounter + 2; i < Ncols ; i++) {
		const T buffer = inputVectors[i];
    	outputRightHouseholderVectors[i] = buffer;
    	// --- Evaluate the squared norm of input vector x
    	norm2 = norm2 + buffer * buffer;
	}

    x0 = inputVectors[iterCounter + 1];

    if(norm2 == 0.) betap = static_cast<T>(0);
    else {
		mu = sqrt(x0 * x0 + norm2);

		if (x0 <= 0)	v0 = x0 - mu;
    	else			v0 = -norm2 / (x0 + mu);

        T temp = static_cast<T>(1) / v0;
    	betap = static_cast<T>(2) * ((v0 * v0) / (norm2 + v0 * v0));
    	outputRightHouseholderVectors[iterCounter + 1] = static_cast<T>(1);
#pragma unroll
		for (unsigned int s = iterCounter + 2; s < Ncols ; s++) outputRightHouseholderVectors[s] *= temp;
	}
}

/*************************************/
/* BIDIAGONALIZATION KERNEL FUNCTION */
/*************************************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Nrows, const unsigned int Ncols>
__global__ void bidiagonalizeKernel(T *inputMatrices, const unsigned int maxIter) {
#else
template<class T>
__global__ void bidiagonalizeKernel(T *inputMatrices, const unsigned int maxIter, const unsigned int numMatrices, const unsigned int Nrows, const unsigned int Ncols) {
#endif

    int tid = threadIdx.x + blockDim.x * blockIdx.x;

#ifdef TEMPLATE
	T leftHouseholderVectors[Nrows];
    T rightHouseholderVectors[Ncols];
    T inputVectorsForRightHouseholderReflections[Ncols];
    T wi[Nrows];
    T xi[Ncols];
    T zi[Ncols];
#else
	T leftHouseholderVectors[maxNrows];
    T rightHouseholderVectors[maxNcols];
    T inputVectorsForRightHouseholderReflections[maxNcols];
    T wi[maxNrows];
    T xi[maxNcols];
    T zi[maxNcols];
#endif
    T betau = static_cast<T>(0), betav = static_cast<T>(0);

#pragma unroll
	for (unsigned int iterCounter = 0; iterCounter < maxIter; iterCounter++) {

		if (iterCounter < Ncols - 2) {

			if (tid < numMatrices) {

				// --- Calculate left Householder vectors
#ifdef TEMPLATE
				computeLeftHouseholderVectors<T, numMatrices, Nrows, Ncols>(inputMatrices, leftHouseholderVectors, betau, tid, iterCounter);
#else
				computeLeftHouseholderVectors(inputMatrices, leftHouseholderVectors, betau, tid, iterCounter, numMatrices, Nrows, Ncols);
#endif
				// --- Calculate xi and input vectors for right Householder reflections
				{
#pragma unroll
					for (unsigned int j = 0; j < Ncols; j++) {
						T sum = static_cast<T>(0);
#pragma unroll
						for (unsigned int i = 0; i < Nrows; i++) sum += -betau * leftHouseholderVectors[i] * inputMatrices[tid + i * numMatrices * Ncols + j * numMatrices];
						inputVectorsForRightHouseholderReflections[j] = sum + inputMatrices[tid + j*numMatrices + iterCounter*numMatrices*Ncols];
						xi[j] = -sum;
					}
				}

				// --- Calculate right Householder vectors
				//computeRightHouseholderVectors(inputVectorsForRightHouseholderReflections, rightHouseholderVectors, betav, tid, iterCounter, numMatrices, Nrows, Ncols);
#ifdef TEMPLATE
				computeRightHouseholderVectors<T, numMatrices, Nrows, Ncols>(inputVectorsForRightHouseholderReflections, rightHouseholderVectors, betav, tid, iterCounter);
#else
				computeRightHouseholderVectors(inputVectorsForRightHouseholderReflections, rightHouseholderVectors, betav, tid, iterCounter, numMatrices, Nrows, Ncols);
#endif
				// --- Calculate wi
				{
#pragma unroll
					for (unsigned int i = 0; i < Nrows; i++) {
						T sum = static_cast<T>(0);
#pragma unroll
						for (unsigned int j = 0; j < Ncols; j++) sum += betav * rightHouseholderVectors[j] * inputMatrices[tid + i * numMatrices * Ncols + j * numMatrices];
						wi[i] = sum;
					}
				}

				// --- Calculate zi
				{
					//T temp	= static_cast<T>(0);
					T sum	= static_cast<T>(0);
#pragma unroll
					for(unsigned int i = 0; i < Ncols; i++) sum += xi[i] * rightHouseholderVectors[i];
					//temp = sum;
					//for (unsigned int i = 0; i < Ncols; i++) zi[i] = xi[i] - betav * temp * rightHouseholderVectors[i];
#pragma unroll
					for (unsigned int i = 0; i < Ncols; i++) zi[i] = xi[i] - betav * sum * rightHouseholderVectors[i];
				}

				// --- Matrix update
#pragma unroll
				for (unsigned int r = 0; r < Nrows; r++)
#pragma unroll
					for(unsigned int c = 0; c < Ncols; c++)
						inputMatrices[tid + r * numMatrices * Ncols + c * numMatrices] = -leftHouseholderVectors[r] * zi[c]
						                                                                 -wi[r] * rightHouseholderVectors[c] +
																						  inputMatrices[tid + r * numMatrices * Ncols + c * numMatrices];
			}

		}

		if (iterCounter >= (Ncols - 2)) {

			if (tid < numMatrices) {

				// --- Calculate the Householder vectors
				//computeLeftHouseholderVectors(inputMatrices, leftHouseholderVectors, betau, tid, numMatrices, Nrows, Ncols, iterCounter);
#ifdef TEMPLATE
				computeLeftHouseholderVectors<T, numMatrices, Nrows, Ncols>(inputMatrices, leftHouseholderVectors, betau, tid, iterCounter);
#else
				computeLeftHouseholderVectors(inputMatrices, leftHouseholderVectors, betau, tid, iterCounter, numMatrices, Nrows, Ncols);
#endif
				// --- Calculate the xi
#pragma unroll
				for (unsigned int j = 0; j < Ncols; j++) {
					T sum = static_cast<T>(0);
#pragma unroll
					for (unsigned int i = 0; i < Nrows; i++) sum += leftHouseholderVectors[i] * inputMatrices[tid + i * numMatrices * Ncols + j * numMatrices];
					xi[j] = sum;
				}

				// --- Matrix update
#pragma unroll
				for (unsigned int r = 0; r < Nrows; r++)
#pragma unroll
					for (unsigned int c = 0; c < Ncols; c++)
						inputMatrices[tid + r * numMatrices * Ncols + c * numMatrices] = -betau * xi[c] * leftHouseholderVectors[r] +
							                                                              inputMatrices[tid + r * numMatrices * Ncols + c * numMatrices];
			}
		}
	}
}

/*****************/
/* BIDIAGONALIZE */
/*****************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Nrows, const unsigned int Ncols, const int blockSizeBidiagonalize,
         const int blockSizeExtractDiagonals>
void bidiagonalize(T *inputMatrices, T *d, T *e) {
#else
template<class T>
void bidiagonalize(T *inputMatrices, T *d, T *e, const unsigned int numMatrices, const unsigned int Nrows, const unsigned int Ncols,
				   const int blockSizeBidiagonalize, const int blockSizeExtractDiagonals) {
#endif

    const unsigned int maxIter = (Nrows == Ncols ? Ncols - 1 : Ncols);

#ifdef TEMPLATE
	gpuErrchk(cudaFuncSetCacheConfig(bidiagonalizeKernel<T, numMatrices, Nrows, Ncols>,	cudaFuncCachePreferL1));
	gpuErrchk(cudaFuncSetCacheConfig(extractDiagonals<T, numMatrices, Nrows, Ncols>,	cudaFuncCachePreferL1));
#else
	gpuErrchk(cudaFuncSetCacheConfig(bidiagonalizeKernel<T>,	cudaFuncCachePreferL1));
	gpuErrchk(cudaFuncSetCacheConfig(extractDiagonals<T>,		cudaFuncCachePreferL1));
#endif

	// --- ??? Questo ciclo for si può portare nel kernel?
	//for (unsigned int iterCounter = 0; iterCounter < maxIter ; iterCounter++) {
 //
 //       //bidiagonalizeKernel<<<iDivUp(numMatrices, BLOCKSIZE), BLOCKSIZE>>>(inputMatrices, numMatrices, Nrows, Ncols, iterCounter);
 //       bidiagonalizeKernel<T, numMatrices, Nrows, Ncols><<<iDivUp(numMatrices, BLOCKSIZE), BLOCKSIZE>>>(inputMatrices, iterCounter);
	//	#ifdef DEBUG
	//	    gpuErrchk(cudaPeekAtLastError());
	//	    gpuErrchk(cudaDeviceSynchronize());
 //       #endif
	//}

#ifdef TEMPLATE
	bidiagonalizeKernel<T, numMatrices, Nrows, Ncols><<<iDivUp(numMatrices, blockSizeBidiagonalize), blockSizeBidiagonalize>>>(inputMatrices, maxIter);
#else
	bidiagonalizeKernel<T><<<iDivUp(numMatrices, blockSizeBidiagonalize), blockSizeBidiagonalize>>>(inputMatrices, maxIter, numMatrices, Nrows, Ncols);
#endif
#ifdef DEBUG_BIDIAGONALIZE
	gpuErrchk(cudaPeekAtLastError());
	gpuErrchk(cudaDeviceSynchronize());
#endif

#ifdef TEMPLATE
	extractDiagonals<T, numMatrices, Nrows, Ncols><<<iDivUp(numMatrices, blockSizeExtractDiagonals), blockSizeExtractDiagonals>>>(inputMatrices, d, e);
#else
	extractDiagonals<<<iDivUp(numMatrices, blockSizeExtractDiagonals), blockSizeExtractDiagonals>>>(inputMatrices, d, e, numMatrices, Nrows, Ncols);
#endif

#ifdef DEBUG_EXTRACT_DIAGONALS
    gpuErrchk(cudaPeekAtLastError());
    gpuErrchk(cudaDeviceSynchronize());
#endif
}

/*******************************/
/* MATRIX REARRANGEMENT KERNEL */
/*******************************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Nrows, const unsigned int Ncols>
__global__ void rearrangeKernel(const T * __restrict__ inputMatrices, T * __restrict__ outputMatrices) {
#else
template<class T>
__global__ void rearrangeKernel(const T * __restrict__ inputMatrices, T * __restrict__ outputMatrices, const unsigned int numMatrices,
	                            const unsigned int Nrows, const unsigned int Ncols) {
#endif

    int tid = threadIdx.x + blockDim.x * blockIdx.x;

    if (tid < numMatrices){

#pragma unroll
		for (unsigned int i = 0; i < Nrows; i++)
#pragma unroll
			for (unsigned int j = 0; j < Ncols; j++)
				outputMatrices[tid + j * numMatrices + i * numMatrices * Ncols] = inputMatrices[tid * Nrows * Ncols + j * Nrows + i];
    }
}

/************************/
/* MATRIX REARRANGEMENT */
/************************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Nrows, const unsigned int Ncols, const int blockSizeRearrange>
void rearrange(const T * __restrict__ inputMatrices, T * __restrict__ outputMatrices) {
#else
template<class T>
void rearrange(const T * __restrict__ inputMatrices, T * __restrict__ outputMatrices, const unsigned int numMatrices, const unsigned int Nrows,
	           const unsigned int Ncols, const int blockSizeRearrange) {
#endif

#ifdef TEMPLATE
	rearrangeKernel<T, numMatrices, Nrows, Ncols><<<iDivUp(numMatrices, blockSizeRearrange), blockSizeRearrange>>>(inputMatrices, outputMatrices);
#else
	rearrangeKernel<T><<<iDivUp(numMatrices, blockSizeRearrange), blockSizeRearrange>>>(inputMatrices, outputMatrices, numMatrices, Nrows, Ncols);
#endif
#ifdef DEBUG_REARRANGE
    gpuErrchk(cudaPeekAtLastError());
    gpuErrchk(cudaDeviceSynchronize());
#endif
}

#endif


'File written in /content/src/step1.h'

In [7]:
%%cuda --name steps2_and_3.h
#ifndef __STEPS2_AND_3_H__
#define __STEPS2_AND_3_H__

#include <cfloat>
#include <cstdio>

//#define DEBUG_STURM_BISECTION

/*****************************/
/* CUSTOMIZED fabs FUNCTIONS */
/*****************************/
__host__ __device__ float  fabsT(float  x) { return fabsf(x); }
__host__ __device__ double fabsT(double x) { return fabs (x); }

/*****************************/
/* CUSTOMIZED fmax FUNCTIONS */
/*****************************/
__host__ __device__ float  fmaxT(float  x, float  y) { return fmaxf(x, y); }
__host__ __device__ double fmaxT(double x, double y) { return fmax (x, y); }

/**********************************************/
/* COMPUTE TRIDIAGONAL MATRIX DEVICE FUNCTION */
/**********************************************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Ncols>
__host__ __device__ __forceinline__ void computeTridiagonalMatrix(const T *d, const T *e, T * __restrict__ alpha, T * __restrict__ beta) {
#else
template<class T>
__host__ __device__ void computeTridiagonalMatrix(const T *d, const T *e, T * __restrict__ alpha, T * __restrict__ beta,
	                                              const unsigned int numMatrices, const unsigned int Ncols){
#endif

    alpha[0]	= d[0] * d[0];
	beta[0]		= d[0] * e[0];

#pragma unroll
	for (unsigned int i = 1; i < Ncols - 1; i++) {
		alpha[i * numMatrices] = e[(i - 1) * numMatrices] * e[(i - 1) * numMatrices] + d[i * numMatrices] * d[i * numMatrices];
        beta [i * numMatrices] = d[i * numMatrices] * e[i * numMatrices];
    }

	if (Ncols > 1) alpha[(Ncols - 1) * numMatrices] = e[(Ncols - 2) * numMatrices] * e[(Ncols - 2) * numMatrices] + d[(Ncols - 1) * numMatrices] * d[(Ncols - 1) * numMatrices];

}

/*****************/
/* COMPUTE PIVOT */
/*****************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Ncols>
__host__ __device__ __forceinline__ void computePivot(const T * __restrict__ b, T * __restrict__ pivot) {
#else
template<class T>
__host__ __device__ __forceinline__ void computePivot(const T * __restrict__ b, T * __restrict__ pivot, const unsigned int numMatrices, const unsigned int Ncols) {
#endif

    const unsigned int length = Ncols - 1;
    T currentMax = b[0];
    for (unsigned int i = 1; i < length; i++) {
        T temp = b[i * numMatrices];
        if (temp > currentMax) currentMax = temp;
    }

    *pivot = (static_cast<T>(2) * fmaxT(static_cast<T>(1), currentMax)) / real_MAX;
}

//#ifdef TEMPLATE
//template<const unsigned int numMatrices, const unsigned int Ncols>
//__host__ __device__ __forceinline__ void computePivot(const double * __restrict__ b, double * __restrict__ pivot) {
//#else
//__host__ __device__ __forceinline__ void computePivot(const double * __restrict__ b, double * __restrict__ pivot, const unsigned int numMatrices, const unsigned int Ncols) {
//#endif
//
//    const unsigned int length = Ncols - 1;
//    double currentMax = b[0];
//    for (unsigned int i = 1; i < length; i++) {
//        double temp = b[i * numMatrices];
//        if (temp > currentMax) currentMax = temp;
//    }
//
//    *pivot = (static_cast<double>(2) * fmax(static_cast<double>(1), currentMax)) / DBL_MAX;
//}
//
//__host__ __device__ __forceinline__ void computePivot(const float * __restrict__ b, float * __restrict__ pivot, const unsigned int Ncols, const unsigned int numMatrices) {
//
//    const unsigned int length = Ncols - 1;
//    float currentMax = b[0];
//    for (unsigned int i = 1; i < length; i++) {
//        float temp = b[i * numMatrices];
//        if (temp > currentMax) currentMax = temp;
//    }
//
//    *pivot = (static_cast<float>(2) * fmaxf(static_cast<float>(1), currentMax)) / FLT_MAX;
//}

/***********************************/
/* COMPUTE INITIAL SEARCH INTERVAL */
/***********************************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Ncols>
__host__ __device__ __forceinline__ void computeInitialInterval(const T * __restrict__ d, const T * __restrict__ b, T &alpha,
																T &beta) {
#else
template<class T>
__host__ __device__ __forceinline__ void computeInitialInterval(const T * __restrict__ d, const T * __restrict__ b, T &alpha,
																T &beta, const unsigned int numMatrices, const unsigned int Ncols) {
#endif

    T temp1 = static_cast<T>(0);
    T temp2 = static_cast<T>(0);

    alpha	= d[0] - fabsT(b[0]);
    beta	= fabsT(d[0]) + fabsT(b[0]);

	temp1 = d[(Ncols - 1) * numMatrices] - fabsT(b[(Ncols - 2) * numMatrices]);
    temp2 = fabsT(d[(Ncols - 1) * numMatrices]) + fabsT(b[(Ncols - 2) * numMatrices]);

	if (temp1 < alpha) alpha = temp1;
    if (temp2 > beta)  beta	 = temp2;

	for (unsigned int i = 1; i < Ncols - 1; i++) {
        temp1 = d[i * numMatrices] - fabsT(b[(i - 1) * numMatrices]) - fabsT(b[i * numMatrices]);
        temp2 = d[i * numMatrices] + fabsT(b[(i - 1) * numMatrices]) + fabsT(b[i * numMatrices]);
        if (temp1 < alpha) alpha = temp1;
        if (temp2 > beta)  beta  = temp2;
    }

}

//#ifdef TEMPLATE
//template<const unsigned int numMatrices, const unsigned int Ncols>
//__host__ __device__ __forceinline__ void computeInitialInterval(const double * __restrict__ d, const double * __restrict__ b, double &alpha,
//																double &beta) {
//#else
//__host__ __device__ __forceinline__ void computeInitialInterval(const double * __restrict__ d, const double * __restrict__ b, double &alpha,
//																double &beta, const unsigned int numMatrices, const unsigned int Ncols) {
//#endif
//
//    double temp1 = static_cast<double>(0);
//    double temp2 = static_cast<double>(0);
//
//    alpha	= d[0] - fabs(b[0]);
//    beta	= fabs(d[0]) + fabs(b[0]);
//
//	temp1 = d[(Ncols - 1) * numMatrices] - fabs(b[(Ncols - 2) * numMatrices]);
//    temp2 = fabs(d[(Ncols - 1) * numMatrices]) + fabs(b[(Ncols - 2) * numMatrices]);
//
//	if (temp1 < alpha) alpha = temp1;
//    if (temp2 > beta)  beta	 = temp2;
//
//	for (unsigned int i = 1; i < Ncols - 1; i++) {
//        temp1 = d[i * numMatrices] - fabs(b[(i - 1) * numMatrices]) - fabs(b[i * numMatrices]);
//        temp2 = d[i * numMatrices] + fabs(b[(i - 1) * numMatrices]) + fabs(b[i * numMatrices]);
//        if (temp1 < alpha) alpha = temp1;
//        if (temp2 > beta)  beta  = temp2;
//    }
//
//}
//
//#ifdef TEMPLATE
//template<const unsigned int numMatrices, const unsigned int Ncols>
//__host__ __device__ __forceinline__ void computeInitialInterval(const float * __restrict__ d, const float * __restrict__ b, float &alpha,
//																float &beta) {
//#else
//__host__ __device__ __forceinline__ void computeInitialInterval(const float * __restrict__ d, const float * __restrict__ b, float &alpha,
//																float &beta, const unsigned int numMatrices, const unsigned int Ncols) {
//#endif
//
//    float temp1 = static_cast<float>(0);
//    float temp2 = static_cast<float>(0);
//
//    alpha	= d[0] - fabsf(b[0]);
//    beta	= fabsf(d[0]) + fabsf(b[0]);
//
//	temp1 = d[(Ncols - 1) * numMatrices] - fabsf(b[(Ncols - 2) * numMatrices]);
//    temp2 = fabsf(d[(Ncols - 1) * numMatrices]) + fabsf(b[(Ncols - 2) * numMatrices]);
//
//	if (temp1 < alpha) alpha = temp1;
//    if (temp2 > beta)  beta	 = temp2;
//
//	for (unsigned int i = 1; i < Ncols - 1; i++) {
//        temp1 = d[i * numMatrices] - fabsf(b[(i - 1) * numMatrices]) - fabsf(b[i * numMatrices]);
//        temp2 = d[i * numMatrices] + fabsf(b[(i - 1) * numMatrices]) + fabsf(b[i * numMatrices]);
//        if (temp1 < alpha) alpha = temp1;
//        if (temp2 > beta)  beta  = temp2;
//    }
//
//}

/*************/
/* CHECKSIGN */
/*************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Ncols>
__host__ __device__ void chcksign(const T * __restrict__ d, const T * __restrict__ b, const T pivot, const T x, unsigned int &numChanges) {
#else
template<class T>

__host__ __device__ void chcksign(const T * __restrict__ d, const T * __restrict__ b, const T pivot, const T x, unsigned int &numChanges,
	                              const unsigned int numMatrices, const unsigned int Ncols) {
#endif

    numChanges = 0;

    T q = d[0] - x;
    if (fabsT(q) <= pivot) q = -pivot;
    if (q < static_cast<T>(0)) numChanges = numChanges + 1;

    for (unsigned int i = 2; i <= Ncols; i++) {
        // --- This order of operation preserve monotonicity
		q = (d[(i - 1) * numMatrices] - ((b[(i - 2) * numMatrices] * b[(i - 2) * numMatrices]) / q)) - x;
        if (fabsT(q) <= pivot) q = -pivot;
        if (q < static_cast<T>(0)) numChanges = numChanges + 1;
    }
}

/************************************/
/* STURM METHOD BASED ON BISECTIONS */
/************************************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Ncols, const int blockSizeSturm>
__global__ void sturmBisection(const T * __restrict__ d, const T * __restrict__ b, T * __restrict__ dd, T * __restrict__ bb,
                               T * __restrict__ singularValues, const T tol){
#else
template<class T, const int blockSizeSturm>
__global__ void sturmBisection(const T * __restrict__ d, const T * __restrict__ b, T * __restrict__ dd, T * __restrict__ bb,
                               T * __restrict__ singularValues, const unsigned int numMatrices, const unsigned int Ncols, const T tol) {
#endif

    int tidx = threadIdx.x + blockDim.x * blockIdx.x;
    int tidy = threadIdx.y;

    T alpha, beta, pivot;

    __shared__ T alpha_shared [blockSizeSturm];
    __shared__ T beta_shared  [blockSizeSturm];
    __shared__ T pivmin_shared[blockSizeSturm];

    if (tidx < numMatrices && tidy == 0) {
#ifdef TEMPLATE
        computeTridiagonalMatrix<T, numMatrices, Ncols>(d + tidx, b + tidx, dd + tidx, bb + tidx);
#else
        computeTridiagonalMatrix(d + tidx, b + tidx, dd + tidx, bb + tidx, numMatrices, Ncols);
#endif
#ifdef TEMPLATE
	    computePivot<T, numMatrices, Ncols>(bb + tidx, &pivmin_shared[threadIdx.x]);
#else
	    computePivot<T>(bb + tidx, &pivmin_shared[threadIdx.x], numMatrices, Ncols);
#endif
#ifdef TEMPLATE
		computeInitialInterval<T, numMatrices, Ncols>(dd + tidx, bb + tidx, alpha_shared[threadIdx.x], beta_shared[threadIdx.x]);
#else
		computeInitialInterval(dd + tidx, bb + tidx, alpha_shared[threadIdx.x], beta_shared[threadIdx.x], numMatrices, Ncols);
#endif
    }
    __syncthreads();

    if (tidx < numMatrices && tidy < Ncols) {

        unsigned int eigIndex = tidy % Ncols;
        //unsigned int eigIndex = tidy & (Ncols - 1);

        unsigned int	numChanges = 0;
        T				c		   = 0;

        alpha	= alpha_shared [threadIdx.x];
        beta	= beta_shared  [threadIdx.x];
        pivot	= pivmin_shared[threadIdx.x];

        T dist	= fabsT(beta - alpha); // --- Initial amplitude of the search interval
        T s		= fabsT(beta) + fabsT(alpha);

        while (dist > tol * s) { // --- Quarteroni
        //while(dist > max(tol, DBL_EPSILON * max(abs(alpha), abs(beta)))) { // --- Demmel - Dhillon

            c = (alpha + beta) / static_cast<T>(2);

#ifdef TEMPLATE
			chcksign<T, numMatrices, Ncols>(dd + tidx, bb + tidx, pivot, c, numChanges);
#else
			chcksign(dd + tidx, bb + tidx, pivot, c, numChanges, numMatrices, Ncols);
#endif

            if (numChanges > (Ncols - (eigIndex + 1)))	beta  = c;
            else										alpha = c;

            dist	= fabsT(beta - alpha);
            s		= fabsT(beta) + fabsT(alpha);
        }

        singularValues[eigIndex + tidx * Ncols] = sqrt(fabsT(c));
        //printf("%f\n", singularValues[eigIndex + tidx * Ncols]);
    }
}

/*********************************************************************/
/* TRIDIAGONALIZE AND FIND THE EIGENVALUES OF THE TRIDIAGONAL MATRIX */
/*********************************************************************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Ncols, const int blockSizeSturm>
void tridiagonalizeAndFindEigenvalues(const T * __restrict__ d, const T * __restrict__ b, T * __restrict__ dd, T * __restrict__ bb,
	                                  T * __restrict__ singularValues, const T tol) {
#else
template<class T, const int blockSizeSturm>
void tridiagonalizeAndFindEigenvalues(const T * __restrict__ d, const T * __restrict__ b, T * __restrict__ dd, T * __restrict__ bb,
	                                  T * __restrict__ singularValues, const unsigned int numMatrices, const unsigned int Ncols, const T tol) {
#endif

	const unsigned int NUM_BLOCKS_STURM = iDivUp(numMatrices, blockSizeSturm);
    dim3 BLOCKGRID(blockSizeSturm, Ncols);

#ifdef TEMPLATE
	gpuErrchk(cudaFuncSetCacheConfig(sturmBisection<T, numMatrices, Ncols, blockSizeSturm>,	cudaFuncCachePreferL1));
#else
	gpuErrchk(cudaFuncSetCacheConfig(sturmBisection<T, blockSizeSturm>,						cudaFuncCachePreferL1));
#endif

#ifdef TEMPLATE
	sturmBisection<T, numMatrices, Ncols, blockSizeSturm><<<NUM_BLOCKS_STURM, BLOCKGRID>>>(d, b, dd, bb, singularValues, tol);
#else
	sturmBisection<T, blockSizeSturm><<<NUM_BLOCKS_STURM, BLOCKGRID>>>(d, b, dd, bb, singularValues, numMatrices, Ncols, tol);
#endif
#ifdef DEBUG_STURM_BISECTION
    gpuErrchk(cudaPeekAtLastError());
    gpuErrchk(cudaDeviceSynchronize());
#endif

}

#endif


'File written in /content/src/steps2_and_3.h'

In [8]:
%%cuda --name sv.h
#ifndef __SV_H__
#define __SV_H__

#include "step1.h"
#include "steps2_and_3.h"
#include <time.h>
#include <iomanip>

#include "TimingGPU.h"

/**********************/
/* SVD PLAN STRUCTURE */
/**********************/
template<class T>
struct svdPlan {
	T *dev_mat_opt;
    T *dev_mat;
	T *dev_diag;		// --- Diagonal of bidiagonal
	T *dev_supdiag;		// --- Off-diagonal of bidiagonal
	T *alpha;			// --- Diagonal of tridiagonal
	T *beta;			// --- Off-diagonal of tridiagonal
	T *singularValues;	// --- Singular values
};

/*********************/
/* SVD PLAN CREATION */
/*********************/
template<class T>
void createPlan(svdPlan<T>& plan, unsigned int Nrows, unsigned int Ncols, unsigned int numMatrices, unsigned int gpuID) {

    // --- Device allocations
    gpuErrchk(cudaSetDevice(gpuID));
	gpuErrchk(cudaMalloc(&(plan.dev_mat_opt),		Nrows	*	Ncols		*	numMatrices * sizeof(T)));
	gpuErrchk(cudaMalloc(&(plan.dev_mat),			Nrows   *   Ncols		*   numMatrices * sizeof(T)));
 	gpuErrchk(cudaMalloc(&(plan.dev_diag),			            Ncols		*   numMatrices * sizeof(T)));
    gpuErrchk(cudaMalloc(&(plan.dev_supdiag),		            (Ncols - 1) *	numMatrices * sizeof(T)));
    gpuErrchk(cudaMalloc(&(plan.alpha),							Ncols       *   numMatrices * sizeof(T)));
	gpuErrchk(cudaMalloc(&(plan.beta),				            (Ncols - 1) *   numMatrices * sizeof(T)));
	gpuErrchk(cudaMalloc(&(plan.singularValues),	            Ncols 		*   numMatrices * sizeof(T)));

}

/************************/
/* SVD PLAN DESTRUCTION */
/************************/
template<class real_type>
void destroyPlan(svdPlan<real_type>& plan, unsigned int gpuID) {

    // --- Device deallocations
    gpuErrchk(cudaSetDevice(gpuID));
    gpuErrchk(cudaFree(plan.dev_mat_opt));
	gpuErrchk(cudaFree(plan.dev_mat));
	gpuErrchk(cudaFree(plan.dev_diag));
    gpuErrchk(cudaFree(plan.dev_supdiag));
    gpuErrchk(cudaFree(plan.beta));
	gpuErrchk(cudaFree(plan.alpha));
	gpuErrchk(cudaFree(plan.singularValues));
}

/*********************************/
/* HOST-SIDE COMPUTATION ROUTINE */
/*********************************/
#ifdef TEMPLATE
template<class T, const unsigned int numMatrices, const unsigned int Nrows, const unsigned int Ncols, const int blockSizeBidiagonalize,
         const int blockSizeExtractDiagonals, const int blockSizeRearrange, const int blockSizeSturm>
void my_svd(svdPlan<T> *plan, T *inputMatrices, double &rearrangeTime, double &bidiagonalizationTime,
	        double &tridiagAndBisectionTime, double &hostToDeviceTime, const T tol) {
#else
template<class T, const int blockSizeSturm>
void my_svd(svdPlan<T> *plan, T *inputMatrices, double &rearrangeTime, double &bidiagonalizationTime,
	        double &tridiagAndBisectionTime, double &hostToDeviceTime, const unsigned int numMatrices, const unsigned int Nrows,
			const unsigned int Ncols, const T tol, const int blockSizeBidiagonalize, const int blockSizeExtractDiagonals, const int blockSizeRearrange) {
#endif

#ifdef DEBUG
	TimingGPU timer;
#endif

	// --- Host -> Device memory transfers
#ifdef DEBUG
	timer.StartCounter();
#endif
	for (int k = 0; k < numGPUs; k++) {
		gpuErrchk(cudaSetDevice(k));
		gpuErrchk(cudaMemcpyAsync(plan[k].dev_mat, inputMatrices + k * Nrows * Ncols * numMatrices, Nrows * Ncols * numMatrices * sizeof(T), cudaMemcpyHostToDevice));
	}
#ifdef DEBUG
	hostToDeviceTime	+= timer.GetCounter();
#endif

	// --- Reorganize the input matrix
#ifdef DEBUG
	timer.StartCounter();
#endif
#ifdef TEMPLATE
	for (int k = 0; k < numGPUs; k++) {
		gpuErrchk(cudaSetDevice(k));
		rearrange<T, numMatrices, Nrows, Ncols, blockSizeRearrange>(plan[k].dev_mat, plan[k].dev_mat_opt);
	}
#else
	for (int k = 0; k < numGPUs; k++) {
		gpuErrchk(cudaSetDevice(k));
		rearrange(plan[k].dev_mat, plan[k].dev_mat_opt, numMatrices, Nrows, Ncols, blockSizeRearrange);
	}
#endif
#ifdef DEBUG
	rearrangeTime	+= timer.GetCounter();
#endif

	// --- Compute bidiagonalization
#ifdef DEBUG
	timer.StartCounter();
#endif
#ifdef TEMPLATE
	for (int k = 0; k < numGPUs; k++) {
		gpuErrchk(cudaSetDevice(k));
		bidiagonalize<T, numMatrices, Nrows, Ncols, blockSizeBidiagonalize, blockSizeExtractDiagonals>(plan[k].dev_mat_opt, plan[k].dev_diag, plan[k].dev_supdiag);
	}
#else
	for (int k = 0; k < numGPUs; k++) {
		gpuErrchk(cudaSetDevice(k));
		bidiagonalize(plan[k].dev_mat_opt, plan[k].dev_diag, plan[k].dev_supdiag, numMatrices, Nrows, Ncols, blockSizeBidiagonalize, blockSizeExtractDiagonals);
	}
#endif
#ifdef DEBUG
	bidiagonalizationTime += timer.GetCounter();
#endif

	// --- Compute tridiagonal matrix and find eigenvalues
#ifdef DEBUG
	timer.StartCounter();
#endif
#ifdef TEMPLATE
	for (int k = 0; k < numGPUs; k++) {
		gpuErrchk(cudaSetDevice(k));
		tridiagonalizeAndFindEigenvalues<T, numMatrices, Ncols, blockSizeSturm>(plan[k].dev_diag, plan[k].dev_supdiag, plan[k].alpha, plan[k].beta, plan[k].singularValues, tol);
	}
#else
	for (int k = 0; k < numGPUs; k++) {
		gpuErrchk(cudaSetDevice(k));
		tridiagonalizeAndFindEigenvalues<T, blockSizeSturm>(plan[k].dev_diag, plan[k].dev_supdiag, plan[k].alpha, plan[k].beta, plan[k].singularValues, numMatrices, Ncols, tol);
	}
#endif
#ifdef DEBUG
	tridiagAndBisectionTime	+= timer.GetCounter();
#endif

}

#endif


'File written in /content/src/sv.h'

In [9]:
%%cuda --name TimingGPU.cu
/**************/
/* TIMING GPU */
/**************/

#include "TimingGPU.h"

#include <cuda.h>
#include <cuda_runtime.h>

struct PrivateTimingGPU {
	cudaEvent_t		start;
	cudaEvent_t		stop;
};

// default constructor
TimingGPU::TimingGPU() { privateTimingGPU = new PrivateTimingGPU; }

// default destructor
TimingGPU::~TimingGPU() { }

void TimingGPU::StartCounter()
{
	cudaEventCreate(&((*privateTimingGPU).start));
	cudaEventCreate(&((*privateTimingGPU).stop));
	cudaEventRecord((*privateTimingGPU).start, 0);
}

void TimingGPU::StartCounterFlags()
{
	int eventflags = cudaEventBlockingSync;

	cudaEventCreateWithFlags(&((*privateTimingGPU).start), eventflags);
	cudaEventCreateWithFlags(&((*privateTimingGPU).stop), eventflags);
	cudaEventRecord((*privateTimingGPU).start, 0);
}

// Gets the counter in ms
float TimingGPU::GetCounter()
{
	float	time;
	cudaEventRecord((*privateTimingGPU).stop, 0);
	cudaEventSynchronize((*privateTimingGPU).stop);
	cudaEventElapsedTime(&time, (*privateTimingGPU).start, (*privateTimingGPU).stop);
	return time;
}


'File written in /content/src/TimingGPU.cu'

In [10]:
%%cuda --name TimingGPU.h
#ifndef __TIMINGGPU_CUH__
#define __TIMINGGPU_CUH__

/**************/
/* TIMING GPU */
/**************/

// Events are a part of CUDA API and provide a system independent way to measure execution times on CUDA devices with approximately 0.5
// microsecond precision.

struct PrivateTimingGPU;

class TimingGPU
{
private:
	PrivateTimingGPU *privateTimingGPU;

public:

	TimingGPU();

	~TimingGPU();

	void StartCounter();
	void StartCounterFlags();

	float GetCounter();

}; // TimingCPU class

#endif

'File written in /content/src/TimingGPU.h'

In [11]:
%%cuda --name Utilities.h
#ifndef UTILITIES_CUH
#define UTILITIES_CUH

#include <cusolverDn.h>
#include <cublas_v2.h>
#include <cusparse_v2.h>
#include <curand.h>
#include <cufft.h>

//#include <thrust/pair.h>

//extern "C" int iDivUp(int, int);
__host__ __device__ int iDivUp(int, int);
extern "C" void gpuErrchk(cudaError_t);
extern "C" void cusolveSafeCall(cusolverStatus_t);
extern "C" void cublasSafeCall(cublasStatus_t);
extern "C" void cufftSafeCall(cufftResult err);
extern "C" void cusparseSafeCall(cusparseStatus_t err);
extern "C" void curandSafeCall(curandStatus_t err);

template <class T>
void reverseArray(const T * __restrict__, T * __restrict__, const int, const T a = static_cast<T>(1));

//template <class T>
//thrust::pair<T *,T *> Cartesian2Polar(const T * __restrict__ d_x, const T * __restrict__ d_y, const int N, const T a = static_cast<T>(1));
//
//template <class T>
//thrust::pair<T *,T *> h_Cartesian2Polar(const T * __restrict__ d_x, const T * __restrict__ d_y, const int N, const T a = static_cast<T>(1));

template<class T>
T h_l2_norm(T *v1, T *v2, const int N);

template <class T>
void linearCombination(const T * __restrict__, const T * __restrict__, T * __restrict__, const int, const int, const cublasHandle_t);

void linearCombination(const float * __restrict__, const float * __restrict__, float * __restrict__,
	const int, const int, const cublasHandle_t);

void linearCombination(const double * __restrict__, const double * __restrict__, double * __restrict__,
	const int, const int, const cublasHandle_t);

template<class T>
void vectorAddConstant(T * __restrict__, const T, const int);

template<class T>
void vectorMulConstant(T * __restrict__, const T, const int);

template<class T>
void h_vectorMulConstant(T * __restrict__, const T, const int);

template<class T>
__host__ __device__ T fma2(T, T, T);

__device__ int modulo(int, int);

__device__ double atomicAdd(double *, double);
__device__ float  atomicMin(float *, float);

double deg2rad(double);

void cudaMemoryUsage();

/**************************/
/* TEMPLATE SHARED MEMORY */
/**************************/
// --- Credit to the simpleTemplates CUDA sample
template <typename T>
struct SharedMemory
{
	// Ensure that we won't compile any un-specialized types
	__device__ T *getPointer()
	{
		extern __device__ void error(void);
		error();
		return NULL;
	}
};

// Following are the specializations for the following types.
// int, uint, char, uchar, short, ushort, long, ulong, bool, float, and double
// One could also specialize it for user-defined types.

template <>
struct SharedMemory <int>
{
	__device__ int *getPointer()
	{
		extern __shared__ int s_int[];
		return s_int;
	}
};

template <>
struct SharedMemory <unsigned int>
{
	__device__ unsigned int *getPointer()
	{
		extern __shared__ unsigned int s_uint[];
		return s_uint;
	}
};

template <>
struct SharedMemory <char>
{
	__device__ char *getPointer()
	{
		extern __shared__ char s_char[];
		return s_char;
	}
};

template <>
struct SharedMemory <unsigned char>
{
	__device__ unsigned char *getPointer()
	{
		extern __shared__ unsigned char s_uchar[];
		return s_uchar;
	}
};

template <>
struct SharedMemory <short>
{
	__device__ short *getPointer()
	{
		extern __shared__ short s_short[];
		return s_short;
	}
};

template <>
struct SharedMemory <unsigned short>
{
	__device__ unsigned short *getPointer()
	{
		extern __shared__ unsigned short s_ushort[];
		return s_ushort;
	}
};

template <>
struct SharedMemory <long>
{
	__device__ long *getPointer()
	{
		extern __shared__ long s_long[];
		return s_long;
	}
};

template <>
struct SharedMemory <unsigned long>
{
	__device__ unsigned long *getPointer()
	{
		extern __shared__ unsigned long s_ulong[];
		return s_ulong;
	}
};

template <>
struct SharedMemory <bool>
{
	__device__ bool *getPointer()
	{
		extern __shared__ bool s_bool[];
		return s_bool;
	}
};

template <>
struct SharedMemory <float>
{
	__device__ float *getPointer()
	{
		extern __shared__ float s_float[];
		return s_float;
	}
};

template <>
struct SharedMemory <double>
{
	__device__ double *getPointer()
	{
		extern __shared__ double s_double[];
		return s_double;
	}
};

#endif

'File written in /content/src/Utilities.h'

In [12]:
%%cuda --name Utilities.cu
#include <stdio.h>
#include <assert.h>
//#include <math.h>

#include "cuda_runtime.h"
#include <cuda.h>

#include <cusolverDn.h>
#include <cublas_v2.h>
#include <cufft.h>

#include "Utilities.h"

#define DEBUG

#define PI_R         3.14159265358979323846f

/*******************/
/* iDivUp FUNCTION */
/*******************/
//extern "C" int iDivUp(int a, int b){ return ((a % b) != 0) ? (a / b + 1) : (a / b); }
__host__ __device__ int iDivUp(int a, int b) { return ((a % b) != 0) ? (a / b + 1) : (a / b); }

/********************/
/* CUDA ERROR CHECK */
/********************/
// --- Credit to http://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true)
{
	if (code != cudaSuccess)
	{
		fprintf(stderr, "GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
		if (abort) { exit(code); }
	}
}

extern "C" void gpuErrchk(cudaError_t ans) { gpuAssert((ans), __FILE__, __LINE__); }

/**************************/
/* CUSOLVE ERROR CHECKING */
/**************************/
static const char *_cusolverGetErrorEnum(cusolverStatus_t error)
{
	switch (error)
	{
	case CUSOLVER_STATUS_SUCCESS:
		return "CUSOLVER_SUCCESS";

	case CUSOLVER_STATUS_NOT_INITIALIZED:
		return "CUSOLVER_STATUS_NOT_INITIALIZED";

	case CUSOLVER_STATUS_ALLOC_FAILED:
		return "CUSOLVER_STATUS_ALLOC_FAILED";

	case CUSOLVER_STATUS_INVALID_VALUE:
		return "CUSOLVER_STATUS_INVALID_VALUE";

	case CUSOLVER_STATUS_ARCH_MISMATCH:
		return "CUSOLVER_STATUS_ARCH_MISMATCH";

	case CUSOLVER_STATUS_EXECUTION_FAILED:
		return "CUSOLVER_STATUS_EXECUTION_FAILED";

	case CUSOLVER_STATUS_INTERNAL_ERROR:
		return "CUSOLVER_STATUS_INTERNAL_ERROR";

	case CUSOLVER_STATUS_MATRIX_TYPE_NOT_SUPPORTED:
		return "CUSOLVER_STATUS_MATRIX_TYPE_NOT_SUPPORTED";

	}

	return "<unknown>";
}

inline void __cusolveSafeCall(cusolverStatus_t err, const char *file, const int line)
{
	if (CUSOLVER_STATUS_SUCCESS != err) {
		fprintf(stderr, "CUSOLVE error in file '%s', line %d, error: %s \nterminating!\n", __FILE__, __LINE__, \
			_cusolverGetErrorEnum(err)); \
			assert(0); \
	}
}

extern "C" void cusolveSafeCall(cusolverStatus_t err) { __cusolveSafeCall(err, __FILE__, __LINE__); }

/*************************/
/* CUBLAS ERROR CHECKING */
/*************************/
static const char *_cublasGetErrorEnum(cublasStatus_t error)
{
	switch (error)
	{
	case CUBLAS_STATUS_SUCCESS:
		return "CUBLAS_STATUS_SUCCESS";

	case CUBLAS_STATUS_NOT_INITIALIZED:
		return "CUBLAS_STATUS_NOT_INITIALIZED";

	case CUBLAS_STATUS_ALLOC_FAILED:
		return "CUBLAS_STATUS_ALLOC_FAILED";

	case CUBLAS_STATUS_INVALID_VALUE:
		return "CUBLAS_STATUS_INVALID_VALUE";

	case CUBLAS_STATUS_ARCH_MISMATCH:
		return "CUBLAS_STATUS_ARCH_MISMATCH";

	case CUBLAS_STATUS_MAPPING_ERROR:
		return "CUBLAS_STATUS_MAPPING_ERROR";

	case CUBLAS_STATUS_EXECUTION_FAILED:
		return "CUBLAS_STATUS_EXECUTION_FAILED";

	case CUBLAS_STATUS_INTERNAL_ERROR:
		return "CUBLAS_STATUS_INTERNAL_ERROR";

	case CUBLAS_STATUS_NOT_SUPPORTED:
		return "CUBLAS_STATUS_NOT_SUPPORTED";

	case CUBLAS_STATUS_LICENSE_ERROR:
		return "CUBLAS_STATUS_LICENSE_ERROR";
	}

	return "<unknown>";
}

inline void __cublasSafeCall(cublasStatus_t err, const char *file, const int line)
{
	if (CUBLAS_STATUS_SUCCESS != err) {
		fprintf(stderr, "CUBLAS error in file '%s', line %d, error: %s\nterminating!\n", __FILE__, __LINE__, \
			_cublasGetErrorEnum(err)); \
			assert(0); \
	}
}

extern "C" void cublasSafeCall(cublasStatus_t err) { __cublasSafeCall(err, __FILE__, __LINE__); }

/************************/
/* CUFFT ERROR CHECKING */
/************************/
// See http://stackoverflow.com/questions/16267149/cufft-error-handling
static const char *_cudaGetErrorEnum(cufftResult error)
{
	switch (error)
	{
	case CUFFT_SUCCESS:
		return "CUFFT_SUCCESS - The cuFFT operation was successful";

	case CUFFT_INVALID_PLAN:
		return "CUFFT_INVALID_PLAN - cuFFT was passed an invalid plan handle";

	case CUFFT_ALLOC_FAILED:
		return "CUFFT_ALLOC_FAILED - cuFFT failed to allocate GPU or CPU memory";

	case CUFFT_INVALID_TYPE:
		return "CUFFT_INVALID_TYPE - No longer used";

	case CUFFT_INVALID_VALUE:
		return "CUFFT_INVALID_VALUE - User specified an invalid pointer or parameter";

	case CUFFT_INTERNAL_ERROR:
		return "CUFFT_INTERNAL_ERROR - Driver or internal cuFFT library error";

	case CUFFT_EXEC_FAILED:
		return "CUFFT_EXEC_FAILED - Failed to execute an FFT on the GPU";

	case CUFFT_SETUP_FAILED:
		return "CUFFT_SETUP_FAILED - The cuFFT library failed to initialize";

	case CUFFT_INVALID_SIZE:
		return "CUFFT_INVALID_SIZE - User specified an invalid transform size";

	case CUFFT_UNALIGNED_DATA:
		return "CUFFT_UNALIGNED_DATA - No longer used";

	case CUFFT_INCOMPLETE_PARAMETER_LIST:
		return "CUFFT_INCOMPLETE_PARAMETER_LIST - Missing parameters in call";

	case CUFFT_INVALID_DEVICE:
		return "CUFFT_INVALID_DEVICE - Execution of a plan was on different GPU than plan creation";

	case CUFFT_PARSE_ERROR:
		return "CUFFT_PARSE_ERROR - Internal plan database error";

	case CUFFT_NO_WORKSPACE:
		return "CUFFT_NO_WORKSPACE - No workspace has been provided prior to plan execution";

	case CUFFT_NOT_IMPLEMENTED:
		return "CUFFT_NOT_IMPLEMENTED - Function does not implement functionality for parameters given";

	case CUFFT_LICENSE_ERROR:
		return "CUFFT_LICENSE_ERROR - Used in previous versions";

	case CUFFT_NOT_SUPPORTED:
		return "CUFFT_NOT_SUPPORTED - Operation is not supported for parameters given";
	}

	return "<unknown>";
}

// --- CUFFTSAFECALL
inline void cufftAssert(cufftResult err, const char *file, const int line, bool abort = true)
{
	if (CUFFT_SUCCESS != err) {
		fprintf(stderr, "CUFFTassert: Error nr. %d - %s %s %d\n", err, _cudaGetErrorEnum(err), __FILE__, __LINE__);
		if (abort) exit(err);
	}
}

extern "C" void cufftSafeCall(cufftResult err) { cufftAssert(err, __FILE__, __LINE__); }

/***************************/
/* CUSPARSE ERROR CHECKING */
/***************************/
static const char *_cusparseGetErrorEnum(cusparseStatus_t error)
{
	switch (error)
	{

	case CUSPARSE_STATUS_SUCCESS:
		return "CUSPARSE_STATUS_SUCCESS";

	case CUSPARSE_STATUS_NOT_INITIALIZED:
		return "CUSPARSE_STATUS_NOT_INITIALIZED";

	case CUSPARSE_STATUS_ALLOC_FAILED:
		return "CUSPARSE_STATUS_ALLOC_FAILED";

	case CUSPARSE_STATUS_INVALID_VALUE:
		return "CUSPARSE_STATUS_INVALID_VALUE";

	case CUSPARSE_STATUS_ARCH_MISMATCH:
		return "CUSPARSE_STATUS_ARCH_MISMATCH";

	case CUSPARSE_STATUS_MAPPING_ERROR:
		return "CUSPARSE_STATUS_MAPPING_ERROR";

	case CUSPARSE_STATUS_EXECUTION_FAILED:
		return "CUSPARSE_STATUS_EXECUTION_FAILED";

	case CUSPARSE_STATUS_INTERNAL_ERROR:
		return "CUSPARSE_STATUS_INTERNAL_ERROR";

	case CUSPARSE_STATUS_MATRIX_TYPE_NOT_SUPPORTED:
		return "CUSPARSE_STATUS_MATRIX_TYPE_NOT_SUPPORTED";

	case CUSPARSE_STATUS_ZERO_PIVOT:
		return "CUSPARSE_STATUS_ZERO_PIVOT";
	}

	return "<unknown>";
}

inline void __cusparseSafeCall(cusparseStatus_t err, const char *file, const int line)
{
	if (CUSPARSE_STATUS_SUCCESS != err) {
		fprintf(stderr, "CUSPARSE error in file '%s', line %d, error %s\nterminating!\n", __FILE__, __LINE__, \
			_cusparseGetErrorEnum(err)); \
			assert(0); \
	}
}

extern "C" void cusparseSafeCall(cusparseStatus_t err) { __cusparseSafeCall(err, __FILE__, __LINE__); }

/*************************/
/* CURAND ERROR CHECKING */
/*************************/
static const char *_curandGetErrorEnum(curandStatus_t error)
{
	switch (error)
	{
	case CURAND_STATUS_SUCCESS:
		return "CURAND_SUCCESS";

	case CURAND_STATUS_VERSION_MISMATCH:
		return "CURAND_STATUS_VERSION_MISMATCH";

	case CURAND_STATUS_NOT_INITIALIZED:
		return "CURAND_STATUS_NOT_INITIALIZED";

	case CURAND_STATUS_ALLOCATION_FAILED:
		return "CURAND_STATUS_ALLOCATION_FAILED";

	case CURAND_STATUS_TYPE_ERROR:
		return "CURAND_STATUS_TYPE_ERROR";

	case CURAND_STATUS_OUT_OF_RANGE:
		return "CURAND_STATUS_OUT_OF_RANGE";

	case CURAND_STATUS_LENGTH_NOT_MULTIPLE:
		return "CURAND_STATUS_LENGTH_NOT_MULTIPLE";

	case CURAND_STATUS_DOUBLE_PRECISION_REQUIRED:
		return "CURAND_STATUS_DOUBLE_PRECISION_REQUIRED";

	case CURAND_STATUS_LAUNCH_FAILURE:
		return "CURAND_STATUS_LAUNCH_FAILURE";

	case CURAND_STATUS_PREEXISTING_FAILURE:
		return "CURAND_STATUS_PREEXISTING_FAILURE";

	case CURAND_STATUS_INITIALIZATION_FAILED:
		return "CURAND_STATUS_INITIALIZATION_FAILED";

	case CURAND_STATUS_ARCH_MISMATCH:
		return "CURAND_STATUS_ARCH_MISMATCH";

	case CURAND_STATUS_INTERNAL_ERROR:
		return "CURAND_STATUS_INTERNAL_ERROR";

	}

	return "<unknown>";
}

inline void __curandSafeCall(curandStatus_t err, const char *file, const int line)
{
	if (CURAND_STATUS_SUCCESS != err) {
		fprintf(stderr, "CURAND error in file '%s', line %d, error: %s \nterminating!\n", __FILE__, __LINE__, \
			_curandGetErrorEnum(err)); \
			assert(0); \
	}
}

extern "C" void curandSafeCall(curandStatus_t err) { __curandSafeCall(err, __FILE__, __LINE__); }

/************************/
/* REVERSE ARRAY KERNEL */
/************************/
#define BLOCKSIZE_REVERSE	256

// --- Credit to http://www.drdobbs.com/parallel/cuda-supercomputing-for-the-masses-part/208801731?pgno=2
template <class T>
__global__ void reverseArrayKernel(const T * __restrict__ d_in, T * __restrict__ d_out, const int N, const T a)
{
	// --- Credit to the simpleTemplates CUDA sample
	SharedMemory<T> smem;
	T* s_data = smem.getPointer();

	const int tid = blockDim.x * blockIdx.x + threadIdx.x;
	const int id = threadIdx.x;
	const int offset = blockDim.x * (blockIdx.x + 1);

	// --- Load one element per thread from device memory and store it *in reversed order* into shared memory
	if (tid < N) s_data[BLOCKSIZE_REVERSE - (id + 1)] = a * d_in[tid];

	// --- Block until all threads in the block have written their data to shared memory
	__syncthreads();

	// --- Write the data from shared memory in forward order
	if ((N - offset + id) >= 0) d_out[N - offset + id] = s_data[threadIdx.x];
}

/************************/
/* REVERSE ARRAY KERNEL */
/************************/
template <class T>
void reverseArray(const T * __restrict__ d_in, T * __restrict__ d_out, const int N, const T a) {

	reverseArrayKernel << <iDivUp(N, BLOCKSIZE_REVERSE), BLOCKSIZE_REVERSE, BLOCKSIZE_REVERSE * sizeof(T) >> >(d_in, d_out, N, a);
#ifdef DEBUG
	gpuErrchk(cudaPeekAtLastError());
	gpuErrchk(cudaDeviceSynchronize());
#endif

}

template void reverseArray<float>(const float  * __restrict__, float  * __restrict__, const int, const float);
template void reverseArray<double>(const double * __restrict__, double * __restrict__, const int, const double);

/********************************************************/
/* CARTESIAN TO POLAR COORDINATES TRANSFORMATION KERNEL */
/********************************************************/
#define BLOCKSIZE_CART2POL	256

template <class T>
__global__ void Cartesian2PolarKernel(const T * __restrict__ d_x, const T * __restrict__ d_y, T * __restrict__ d_rho, T * __restrict__ d_theta,
	const int N, const T a) {

	const int tid = blockIdx.x * blockDim.x + threadIdx.x;

	if (tid < N) {
		d_rho[tid] = a * hypot(d_x[tid], d_y[tid]);
		d_theta[tid] = atan2(d_y[tid], d_x[tid]);
	}

}

/*******************************************************/
/* CARTESIAN TO POLAR COORDINATES TRANSFORMATION - GPU */
/*******************************************************/
//template <class T>
//thrust::pair<T *,T *> Cartesian2Polar(const T * __restrict__ d_x, const T * __restrict__ d_y, const int N, const T a) {
//
//	T *d_rho;	gpuErrchk(cudaMalloc((void**)&d_rho,   N * sizeof(T)));
//	T *d_theta; gpuErrchk(cudaMalloc((void**)&d_theta, N * sizeof(T)));
//
//	Cartesian2PolarKernel<<<iDivUp(N, BLOCKSIZE_CART2POL), BLOCKSIZE_CART2POL>>>(d_x, d_y, d_rho, d_theta, N, a);
//#ifdef DEBUG
//	gpuErrchk(cudaPeekAtLastError());
//	gpuErrchk(cudaDeviceSynchronize());
//#endif
//
//	return thrust::make_pair(d_rho, d_theta);
//}
//
//template thrust::pair<float  *, float  *>  Cartesian2Polar<float>  (const float  *, const float  *, const int, const float);
//template thrust::pair<double *, double *>  Cartesian2Polar<double> (const double *, const double *, const int, const double);

/*******************************************************/
/* CARTESIAN TO POLAR COORDINATES TRANSFORMATION - CPU */
/*******************************************************/
//template <class T>
//thrust::pair<T *,T *> h_Cartesian2Polar(const T * __restrict__ h_x, const T * __restrict__ h_y, const int N, const T a) {
//
//	T *h_rho	= (T *)malloc(N * sizeof(T));
//	T *h_theta	= (T *)malloc(N * sizeof(T));
//
//	for (int i = 0; i < N; i++) {
//		h_rho[i]	= a * hypot(h_x[i], h_y[i]);
//		h_theta[i]	= atan2(h_y[i], h_x[i]);
//	}
//
//	return thrust::make_pair(h_rho, h_theta);
//}
//
//template thrust::pair<float  *, float  *>  h_Cartesian2Polar<float>  (const float  *, const float  *, const int, const float);
//template thrust::pair<double *, double *>  h_Cartesian2Polar<double> (const double *, const double *, const int, const double);

/*******************************/
/* COMPUTE L2 NORM OF A VECTOR */
/*******************************/
template<class T>
T h_l2_norm(T *v1, T *v2, const int N) {

	T norm = (T)0;

	for (int i = 0; i < N; ++i)
	{
		T d = v1[i] - v2[i];
		norm = norm + d * d;
	}

	return sqrt(norm);
}

template float  h_l2_norm<float>(float  *, float  *, const int);
template double h_l2_norm<double>(double *, double *, const int);

/*******************************/
/* LINEAR COMBINATION FUNCTION */
/*******************************/
void linearCombination(const float * __restrict__ d_coeff, const float * __restrict__ d_basis_functions_real, float * __restrict__ d_linear_combination,
	const int N_basis_functions, const int N_sampling_points, const cublasHandle_t handle) {

	float alpha = 1.f;
	float beta = 0.f;
	cublasSafeCall(cublasSgemv(handle, CUBLAS_OP_N, N_sampling_points, N_basis_functions, &alpha, d_basis_functions_real, N_sampling_points,
		d_coeff, 1, &beta, d_linear_combination, 1));

}

void linearCombination(const double * __restrict__ d_coeff, const double * __restrict__ d_basis_functions_real, double * __restrict__ d_linear_combination,
	const int N_basis_functions, const int N_sampling_points, const cublasHandle_t handle) {

	double alpha = 1.;
	double beta = 0.;
	cublasSafeCall(cublasDgemv(handle, CUBLAS_OP_N, N_sampling_points, N_basis_functions, &alpha, d_basis_functions_real, N_sampling_points,
		d_coeff, 1, &beta, d_linear_combination, 1));

}

/******************************/
/* ADD A CONSTANT TO A VECTOR */
/******************************/
#define BLOCKSIZE_VECTORADDCONSTANT	256

template<class T>
__global__ void vectorAddConstantKernel(T * __restrict__ d_in, const T scalar, const int N) {

	const int tid = threadIdx.x + blockIdx.x*blockDim.x;

	if (tid < N) d_in[tid] += scalar;

}

template<class T>
void vectorAddConstant(T * __restrict__ d_in, const T scalar, const int N) {

	vectorAddConstantKernel << <iDivUp(N, BLOCKSIZE_VECTORADDCONSTANT), BLOCKSIZE_VECTORADDCONSTANT >> >(d_in, scalar, N);

}

template void  vectorAddConstant<float>(float  * __restrict__, const float, const int);
template void  vectorAddConstant<double>(double * __restrict__, const double, const int);

/*****************************************/
/* MULTIPLY A VECTOR BY A CONSTANT - GPU */
/*****************************************/
#define BLOCKSIZE_VECTORMULCONSTANT	256

template<class T>
__global__ void vectorMulConstantKernel(T * __restrict__ d_in, const T scalar, const int N) {

	const int tid = threadIdx.x + blockIdx.x*blockDim.x;

	if (tid < N) d_in[tid] *= scalar;

}

template<class T>
void vectorMulConstant(T * __restrict__ d_in, const T scalar, const int N) {

	vectorMulConstantKernel << <iDivUp(N, BLOCKSIZE_VECTORMULCONSTANT), BLOCKSIZE_VECTORMULCONSTANT >> >(d_in, scalar, N);

}

template void  vectorMulConstant<float>(float  * __restrict__, const float, const int);
template void  vectorMulConstant<double>(double * __restrict__, const double, const int);

/*****************************************/
/* MULTIPLY A VECTOR BY A CONSTANT - CPU */
/*****************************************/
template<class T>
void h_vectorMulConstant(T * __restrict__ h_in, const T scalar, const int N) {

	for (int i = 0; i < N; i++) h_in[i] *= scalar;

}

template void  h_vectorMulConstant<float>(float  * __restrict__, const float, const int);
template void  h_vectorMulConstant<double>(double * __restrict__, const double, const int);

/*****************************************************/
/* FUSED MULTIPLY ADD OPERATIONS FOR HOST AND DEVICE */
/*****************************************************/
template<class T>
__host__ __device__ T fma2(T x, T y, T z) { return x * y + z; }

template float  fma2<float >(float, float, float);
template double fma2<double>(double, double, double);

/*******************/
/* MODULO FUNCTION */
/*******************/
__device__ int modulo(int val, int _mod)
{
	int P;
	if (val > 0) { (!(_mod & (_mod - 1)) ? P = val&(_mod - 1) : P = val % (_mod)); return P; }
	else
	{
		(!(_mod & (_mod - 1)) ? P = (-val)&(_mod - 1) : P = (-val) % (_mod));
		if (P > 0) return _mod - P;
		else return 0;
	}
}

/***************************************/
/* ATOMIC ADDITION FUNCTION ON DOUBLES */
/***************************************/
#if !defined(__CUDA_ARCH__) || __CUDA_ARCH__ >= 600
#else
__device__ double atomicAdd(double* address, double val)
{
	unsigned long long int* address_as_ull =
		(unsigned long long int*)address;
	register unsigned long long int old = *address_as_ull, assumed;
	do {
		assumed = old;
		old = atomicCAS(address_as_ull, assumed,
			__double_as_longlong(val +
				__longlong_as_double(assumed)));
	} while (assumed != old);
	return __longlong_as_double(old);
}
#endif

/*********************************/
/* ATOMIC MIN FUNCTION ON FLOATS */
/*********************************/
__device__ float atomicMin(float* address, float val)
{
	int* address_as_i = (int*)address;
	int old = *address_as_i, assumed;
	do {
		assumed = old;
		old = ::atomicCAS(address_as_i, assumed,
			__float_as_int(::fminf(val, __int_as_float(assumed))));
	} while (assumed != old);
	return __int_as_float(old);
}

/*********************/
/* DEGREE TO RADIANS */
/*********************/
double deg2rad(double deg) { return deg*PI_R / 180; }

/*********************/
/* CUDA MEMORY USAGE */
/*********************/
void cudaMemoryUsage() {

	size_t free_byte;
	size_t total_byte;

	gpuErrchk(cudaMemGetInfo(&free_byte, &total_byte));

	double free_db = (double)free_byte;
	double total_db = (double)total_byte;
	double used_db = total_db - free_db;

	printf("GPU memory: used = %f, free = %f MB, total available = %f MB\n", used_db / 1024.0 / 1024.0, free_db / 1024.0 / 1024.0, total_db / 1024.0 / 1024.0);

}

'File written in /content/src/Utilities.cu'

In [13]:
%%cuda --name TimingCPU.h
// 1 micro-second accuracy
// Returns the time in seconds

#ifndef __TIMINGCPU_H__
#define __TIMINGCPU_H__

#ifdef __linux__

	class TimingCPU {

		private:
			long cur_time_;

		public:

			TimingCPU();

			~TimingCPU();

			void StartCounter();

			double GetCounter();
	};

#elif _WIN32 || _WIN64

	struct PrivateTimingCPU;

	class TimingCPU
	{
		private:
			PrivateTimingCPU *privateTimingCPU;

		public:

			TimingCPU();

			~TimingCPU();

			void StartCounter();

			double GetCounter();

	}; // TimingCPU class

#endif

#endif


'File written in /content/src/TimingCPU.h'

In [14]:
%%cuda --name TimingCPU.cu
/**************/
/* TIMING CPU */
/**************/

#include "TimingCPU.h"

#ifdef __linux__

	#include <sys/time.h>
	#include <stdio.h>

	TimingCPU::TimingCPU(): cur_time_(0) { StartCounter(); }

	TimingCPU::~TimingCPU() { }

	void TimingCPU::StartCounter()
	{
		struct timeval time;
		if(gettimeofday( &time, 0 )) return;
		cur_time_ = 1000000 * time.tv_sec + time.tv_usec;
	}

	double TimingCPU::GetCounter()
	{
		struct timeval time;
		if(gettimeofday( &time, 0 )) return -1;

		long cur_time = 1000000 * time.tv_sec + time.tv_usec;
		double sec = (cur_time - cur_time_) / 1000000.0;
		if(sec < 0) sec += 86400;
		cur_time_ = cur_time;

		return 1000.*sec;
	}

#elif _WIN32 || _WIN64
	#include <windows.h>
	#include <iostream>

	struct PrivateTimingCPU {
		double	PCFreq;
		__int64 CounterStart;
	};

	// --- Default constructor
	TimingCPU::TimingCPU() { privateTimingCPU = new PrivateTimingCPU; (*privateTimingCPU).PCFreq = 0.0; (*privateTimingCPU).CounterStart = 0; }

	// --- Default destructor
	TimingCPU::~TimingCPU() { }

	// --- Starts the timing
	void TimingCPU::StartCounter()
	{
		LARGE_INTEGER li;
		if(!QueryPerformanceFrequency(&li)) std::cout << "QueryPerformanceFrequency failed!\n";

		(*privateTimingCPU).PCFreq = double(li.QuadPart)/1000.0;

		QueryPerformanceCounter(&li);
		(*privateTimingCPU).CounterStart = li.QuadPart;
	}

	// --- Gets the timing counter in ms
	double TimingCPU::GetCounter()
	{
		LARGE_INTEGER li;
		QueryPerformanceCounter(&li);
		return double(li.QuadPart-(*privateTimingCPU).CounterStart)/(*privateTimingCPU).PCFreq;
	}
#endif


'File written in /content/src/TimingCPU.cu'

In [15]:
%%cuda --name BBComplex.h
#ifndef __BBComplex__H__
#define __BBComplex__H__

#ifndef _WIN32
#define __forceinline __attribute__((always_inline))
#endif
#include "ComplexTypes.h"
#include "Result_Types.h"
#include "Operator_Overloads.h"
#include "Function_Overloads.h"

#endif

'File written in /content/src/BBComplex.h'

In [16]:
%%cuda --name ComplexTypes.h
#ifndef __COMPLEXTYPES_H__
#define __COMPLEXTYPES_H__

#include <iomanip>							// Needed for setw


	#define pi_f  3.14159265358979f					// Greek pi in single precision
	#define pi_d  3.141592653589793238463			// Greek pi in double precision

    class float2_;
	class double2_;

	#define prec_cout 10			// Precision in couting complex numbers

	class int2_ {
	
		public:
			int2 c;

			__host__ __device__ __forceinline __forceinline__ int2_()											{ }
			__host__ __device__ __forceinline __forceinline__ int2_(const int x_,const int y_)					{ c.x=x_;		c.y=y_;}
			__host__ __device__ __forceinline __forceinline__ int2_(const int a)								{ c.x = a;		c.y=0; }
			__host__ __device__ __forceinline __forceinline__ int2_(const float a)								{ c.x = (int)a; c.y=0; }
			__host__ __device__ __forceinline __forceinline__ int2_(const double a)								{ c.x = (int)a; c.y=0; }
			__host__ __device__ __forceinline __forceinline__ int2_(const float2_ a);
			__host__ __device__ __forceinline __forceinline__ int2_(const double2_ a);

			__host__ __device__ __forceinline __forceinline__ const int2_& operator=(const int a)				{ c.x = a;		c.y = 0.;		return *this; }
			__host__ __device__ __forceinline __forceinline__ const int2_& operator=(const float a)				{ c.x = (int)a;	c.y = 0.;		return *this; }
			__host__ __device__ __forceinline __forceinline__ const int2_& operator=(const double a)			{ c.x = (int)a; c.y = 0.;		return *this; }
			__host__ __device__ __forceinline __forceinline__ const int2_& operator=(const int2_ a)				{ c.x = a.c.x;	c.y = a.c.y;	return *this; }
			__host__ __device__ __forceinline __forceinline__ const int2_& operator=(const float2_ a);
			__host__ __device__ __forceinline __forceinline__ const int2_& operator=(const double2_ a);
            __host__ __device__ __forceinline __forceinline__ int2_ operator-()const    {   return int2_(-c.x,-c.y); }									
            __host__ __device__ __forceinline __forceinline__ const int& real()const {return c.x;};                        
            __host__ __device__ __forceinline __forceinline__ int &real() {return c.x;};            
            __host__ __device__ __forceinline __forceinline__ const int& imag()const {return c.y;};                        
            __host__ __device__ __forceinline __forceinline__ int &imag() {return c.y;};            


	};

	class float2_ {

		public:
			float2 c;
			__host__ __device__ __forceinline __forceinline__ float2_(const float x_,const float y_)			{ c.x=x_;				c.y=y_; }
			__host__ __device__ __forceinline __forceinline__ float2_()											{ };
			__host__ __device__ __forceinline __forceinline__ float2_(const int a)								{ c.x = (float)a;		c.y=0.; }
			__host__ __device__ __forceinline __forceinline__ float2_(const float a)							{ c.x = a;				c.y=0.; }
			__host__ __device__ __forceinline __forceinline__ float2_(const double a)							{ c.x = (float)a;		c.y=0.; }
			__host__ __device__ __forceinline __forceinline__ float2_(const int2_ a)							{ c.x = (float)a.c.x;	c.y=(float)a.c.y; }
			__host__ __device__ __forceinline __forceinline__ float2_(const double2_ a);

			__host__ __device__ __forceinline __forceinline__ const float2_& operator=(const int a)				{ c.x = (float)a;		c.y = 0.;			return *this; }
 			__host__ __device__ __forceinline __forceinline__ const float2_& operator=(const float a)			{ c.x = a;				c.y = 0.;			return *this; }
			__host__ __device__ __forceinline __forceinline__ const float2_& operator=(const double a)			{ c.x = (float)a;		c.y = 0.;			return *this; }
			__host__ __device__ __forceinline __forceinline__ const float2_& operator=(const int2_ a)			{ c.x = (float)a.c.x;	c.y = (float)a.c.y;	return *this; }
			__host__ __device__ __forceinline __forceinline__ const float2_& operator=(const float2_ a)			{ c.x = a.c.x;			c.y = a.c.y;		return *this; }
			__host__ __device__ __forceinline __forceinline__ const float2_& operator=(const double2_ a);
            __host__ __device__ __forceinline __forceinline__ float2_ operator-()const    {   return float2_(-c.x,-c.y); }						

            __host__ __device__ __forceinline __forceinline__ const float& real()const {return c.x;};                        
            __host__ __device__ __forceinline __forceinline__ float &real() {return c.x;};            
            __host__ __device__ __forceinline __forceinline__ const float& imag()const {return c.y;};                        
            __host__ __device__ __forceinline __forceinline__ float &imag() {return c.y;};            
			
	};

	class double2_ {
	
		public:
			double2 c;

			__host__ __device__ __forceinline __forceinline__ double2_(const double x_,const double y_)			{ c.x=x_;				c.y=y_; }
			__host__ __device__ __forceinline __forceinline__ double2_()										{ };
			__host__ __device__ __forceinline __forceinline__ double2_(const int a)								{ c.x = (double)a;		c.y=0.; }
			__host__ __device__ __forceinline __forceinline__ double2_(const float a)							{ c.x = (double)a;		c.y=0.; }
			__host__ __device__ __forceinline __forceinline__ double2_(const double a)							{ c.x = a;				c.y=0.; }
			__host__ __device__ __forceinline __forceinline__ double2_(const int2_ a)							{ c.x = (double)a.c.x;	c.y=(double)a.c.y; }
			__host__ __device__ __forceinline __forceinline__ double2_(const float2_ a)							{ c.x = (double)a.c.x;	c.y=(double)a.c.y; }

			__host__ __device__ __forceinline __forceinline__ const double2_& operator=(const int a)			{ c.x = (double)a;		c.y = 0.;				return *this; }
			__host__ __device__ __forceinline __forceinline__ const double2_& operator=(const float a)			{ c.x = (double)a;		c.y = 0.;				return *this; }
			__host__ __device__ __forceinline __forceinline__ const double2_& operator=(const double a)			{ c.x = a;				c.y = 0.;				return *this; }
			__host__ __device__ __forceinline __forceinline__ const double2_& operator=(const int2_ a)			{ c.x = (double)a.c.x;	c.y = (double)a.c.y;	return *this; }
			__host__ __device__ __forceinline __forceinline__ const double2_& operator=(const float2_ a)		{ c.x = (double)a.c.x;	c.y = (double)a.c.y;	return *this; }
			__host__ __device__ __forceinline __forceinline__ const double2_& operator=(const double2_ a)		{ c.x = a.c.x;			c.y = a.c.y;			return *this; }
            __host__ __device__ __forceinline __forceinline__ double2_ operator-()const    {   return double2_(-c.x,-c.y); }			
            
            __host__ __device__ __forceinline __forceinline__ const double& real()const {return c.x;};                        
            __host__ __device__ __forceinline __forceinline__ double &real() {return c.x;};            
            __host__ __device__ __forceinline __forceinline__ const double& imag()const {return c.y;};                        
            __host__ __device__ __forceinline __forceinline__ double &imag() {return c.y;};  

	};

	__host__ __device__ __forceinline __forceinline__ int2_::int2_(const float2_ a)								{ c.x = (int)a.c.x;		c.y=(int)a.c.y; }
	__host__ __device__ __forceinline __forceinline__ int2_::int2_(const double2_ a)						{ c.x = (int)a.c.x;		c.y=(int)a.c.y; }
	__host__ __device__ __forceinline __forceinline__ const int2_& int2_::operator=(const float2_ a)			{ c.x = (int)a.c.x;		c.y = (int)a.c.y;	return *this; }
	__host__ __device__ __forceinline __forceinline__ const int2_& int2_::operator=(const double2_ a)			{ c.x = (int)a.c.x;		c.y = (int)a.c.y;	return *this; }
	__host__ __device__ __forceinline __forceinline__ float2_::float2_(const double2_ a)						{ c.x = (float)a.c.x;	c.y=(float)a.c.y; }
	__host__ __device__ __forceinline __forceinline__ const float2_& float2_::operator=(const double2_ a)		{ c.x = (float)a.c.x;	c.y = (float)a.c.y; return *this; }


	/*std::ostream& operator<<(std::ostream& output, const int2_& v)
	{
		output << std::setw(prec_cout) << "(" << v.c.x << "," << v.c.y << ")\t";
		return output;
	}
	std::ostream& operator<<(std::ostream& output, const float2_& v)
	{
		output << std::setw(prec_cout) << "(" << v.c.x << "," << v.c.y << ")\t";
		return output;
	}
	
	std::ostream& operator<<(std::ostream& output, const double2_& v)
	{
		output << std::setw(prec_cout) << "(" << v.c.x << "," << v.c.y << ")\t";
		return output;
	}*/

#endif

'File written in /content/src/ComplexTypes.h'

In [17]:
%%cuda --name Function_Overloads.h
/*************/
/* REAL PART */
/*************/
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<int>::root_reduced								real(const int a)							{ return a; };
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<float>::root_reduced								real(const float a)							{ return a; };
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<double>::root_reduced							real(const double a)						{ return a; };
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<int2_>::root_reduced								real(const int2_ a)							{ return a.c.x; };
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<float2_>::root_reduced							real(const float2_ a)						{ return a.c.x; };
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<double2_>::root_reduced							real(const double2_ a)						{ return a.c.x; };

/******************/
/* IMAGINARY PART */
/******************/
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<int>::root_reduced								imag(const int a)							{ return 0; }
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<float>::root_reduced								imag(const float a)							{ return 0.f; }
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<double>::root_reduced							imag(const double a)						{ return 0.; }
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<int2_>::root_reduced								imag(const int2_ a)							{ return a.c.y; }
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<float2_>::root_reduced							imag(const float2_ a)						{ return a.c.y; }
__host__ __device__ __forceinline __forceinline__ result_type_root_reduced<double2_>::root_reduced							imag(const double2_ a)						{ return a.c.y; }

/******************/
/* ABSOLUTE VALUE */  //OK
/******************/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions_reduced<int2_>::root_functions_reduced			abs(const int2_ a)							{ return hypot((float)a.c.x,(float)a.c.y); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions_reduced<float2_>::root_functions_reduced		abs(const float2_ a)						{ return hypot(a.c.x,a.c.y); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions_reduced<double2_>::root_functions_reduced		abs(const double2_ a)						{ return hypot(a.c.x,a.c.y); }

/************/
/* ARGUMENT */  //OK
/************/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions_reduced<int>::root_functions_reduced			angle(const int a)							{ return atan2(0.f,(float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions_reduced<float>::root_functions_reduced			angle(const float a)						{ return atan2(0.f,a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions_reduced<double>::root_functions_reduced		angle(const double a)						{ return atan2(0.,a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions_reduced<int2_>::root_functions_reduced			angle(const int2_ a)						{ return atan2((float)a.c.y,(float)a.c.x); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions_reduced<float2_>::root_functions_reduced		angle(const float2_ a)						{ return atan2(a.c.y,a.c.x); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions_reduced<double2_>::root_functions_reduced		angle(const double2_ a)						{ return atan2(a.c.y,a.c.x); }

/*************/
/* CONJUGATE */
/*************/
__host__ __device__ __forceinline __forceinline__ int																		conj(const int a)							{ return a; }
__host__ __device__ __forceinline __forceinline__ float																		conj(const float a)							{ return a; }
__host__ __device__ __forceinline __forceinline__ double																	conj(const double a)						{ return a; }
__host__ __device__ __forceinline __forceinline__ int2_																		conj(const int2_ a)							{ return int2_(a.c.x,-a.c.y); }
__host__ __device__ __forceinline __forceinline__ float2_																	conj(const float2_ a)						{ return float2_(a.c.x,-a.c.y); }
__host__ __device__ __forceinline __forceinline__ double2_																	conj(const double2_ a)						{ return double2_(a.c.x,-a.c.y); }

/*******/
/* LOG */  //OK
/*******/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions							log(const int a)							{ return log((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions							log(const int2_ a)							{ return float2_(0.5f*log((float)(a.c.x*a.c.x)+(float)(a.c.y*a.c.y)),atan2((float)a.c.y,(float)a.c.x)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions						log(const float2_ a)						{ return float2_(0.5f*log(a.c.x*a.c.x+a.c.y*a.c.y),atan2(a.c.y,a.c.x)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions						log(const double2_ a)						{ return double2_(0.5*log(a.c.x*a.c.x+a.c.y*a.c.y),atan2(a.c.y,a.c.x)); }

/*********/
/* LOG10 */  //OK
/*********/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions							log10(const int a)							{ return (log((float)a)/log(10.f)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions							log10(const int2_ a)						{ return float2_(log((float)abs(a))/log(10.f),angle(a)/log(10.f)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions						log10(const float2_ a)						{ return float2_(log(abs(a))/log(10.f),angle(a)/log(10.f)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions						log10(const double2_ a)						{ return double2_(log(abs(a))/log(10.),angle(a)/log(10.)); }

/*********/
/* POLAR */
/*********/
__host__ __device__ __forceinline __forceinline__ result_type_root_augmented<int>::root_augmented							polar(const int a, const int b = 0)			{ return float2_(a*cos((float)b),a*sin((float)b)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_augmented<float>::root_augmented							polar(const float a, const float b = 0)		{ return float2_(a*cos(b),a*sin(b)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_augmented<double>::root_augmented						polar(const double a, const double b = 0)	{ return double2_(a*cos(b),a*sin(b)); }

/********/
/* SQRT */  //OK
/********/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions							sqrt(const int a)							{ return sqrt((float)a); }

__host__ __device__ __forceinline __forceinline__ float2_ sqrt(const int2_& z)
{
	float rr = (float)z.c.x;
	float ii = (float)z.c.y;
	
	float x = fabs(rr); float y = fabs(ii);

	if ((x == 0.f)&(y == 0.f)) return float2_(0.f,0.f);

	float temp = hypot(x,y)+x;
	x = sqrt(0.5f * temp);
	y = sqrt(0.5f * y * y / temp);
	if (ii >= 0.f) { if (rr >= 0.f) return float2_(x, y); else return float2_(y, x); }
	else { if (rr >= 0.f) return float2_(x, -y); else return float2_(y, -x); }
}

__host__ __device__ __forceinline __forceinline__ float2_ sqrt(const float2_& z)
{
	float rr = z.c.x;
	float ii = z.c.y;
	
	float x = fabs(rr); float y = fabs(ii);

	if ((x == 0.f)&(y == 0.f)) return float2_(0.f,0.f);

	float temp = hypot(x,y)+x;
	x = sqrt(0.5f * temp);
	y = sqrt(0.5f * y * y / temp);
	if (ii >= 0.f) { if (rr >= 0.f) return float2_(x, y); else return float2_(y, x); }
	else { if (rr >= 0.f) return float2_(x, -y); else return float2_(y, -x); }
}

__host__ __device__ __forceinline __forceinline__ double2_ sqrt(const double2_& z)
{
	double rr = z.c.x;
	double ii = z.c.y;
	
	double x = fabs(rr); double y = fabs(ii);

	if ((x == 0.f)&(y == 0.f)) return float2_(0.f,0.f);

	float temp = hypot(x,y)+x;
	x = sqrt(0.5 * temp);
	y = sqrt(0.5 * y * y / temp);
	if (ii >= 0.) { if (rr >= 0.) return double2_(x, y); else return double2_(y, x); }
	else { if (rr >= 0.) return double2_(x, -y); else return double2_(y, -x); }
}

/*******/
/* SIN */  //OK
/*******/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions		sin(const int a)				{ return sin((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions		sin(const int2_ a)				{ return float2_(sin((float)a.c.x)*cosh((float)a.c.y),cos((float)a.c.x)*sinh((float)a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions	sin(const float2_ a)			{ return float2_(sin(a.c.x)*cosh(a.c.y),cos(a.c.x)*sinh(a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions	sin(const double2_ a)			{ return double2_(sin(a.c.x)*cosh(a.c.y),cos(a.c.x)*sinh(a.c.y)); }

/*******/
/* COS */  //OK
/*******/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions		cos(const int a)				{ return cos((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions		cos(const int2_ a)				{ return float2_(cos((float)a.c.x)*cosh((float)a.c.y),-sin((float)a.c.x)*sinh((float)a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions	cos(const float2_ a)			{ return float2_(cos(a.c.x)*cosh(a.c.y),-sin(a.c.x)*sinh(a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions	cos(const double2_ a)			{ return double2_(cos(a.c.x)*cosh(a.c.y),-sin(a.c.x)*sinh(a.c.y)); }

/*******/
/* TAN */  //OK
/*******/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions		tan(const int a)				{ return tan((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions		tan(const int2_ a)				{ return float2_(tan((float)a.c.x), tanh((float)a.c.y))/float2_(1.f,-tan((float)a.c.x)*tanh((float)a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions	tan(const float2_ a)			{ return float2_(tan(a.c.x), tanh(a.c.y))/float2_(1.f,-tan(a.c.x)*tanh(a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions	tan(const double2_ a)			{ return double2_(tan(a.c.x), tanh(a.c.y))/double2_(1.,-tan(a.c.x)*tanh(a.c.y)); }

/*******/
/* EXP */  //OK
/*******/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions		exp(const int a)				{ return exp((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions		exp(const int2_ a)				{ return exp((float)a.c.x)*float2_(cos((float)a.c.y),sin((float)a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions	exp(const float2_ a)			{ return exp(a.c.x)*float2_(cos(a.c.y),sin(a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions	exp(const double2_ a)			{ return exp(a.c.x)*double2_(cos(a.c.y),sin(a.c.y)); }

/*******/
/* POW */  //TO BE OPTIMIZED
/*******/
// --- Real-Real
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<int,	float>::strongest	pow(const int a, const float b)				{ return pow(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<int,	double>::strongest	pow(const int a, const double b)			{ return pow(a,b); }

// --- Complex-Complex
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<int2_,	int2_>::strongest				pow(const int2_ a, const int2_ b)			{ if ((a.c.x==0)&&(a.c.y==0)) return int2_(0,0); return exp(b*log(a)); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<int2_,	float2_>::strongest				pow(const int2_ a, const float2_ b)			{ if ((a.c.x==0)&&(a.c.y==0)) return int2_(0,0); return exp(b*log(a)); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<int2_,	double2_>::strongest			pow(const int2_ a, const double2_ b)		{ if ((a.c.x==0)&&(a.c.y==0)) return int2_(0,0); return exp(b*log(a)); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<float2_,	int2_>::strongest				pow(const float2_ a, const int2_ b)			{ if ((a.c.x==0.f)&&(a.c.y==0.f)) return float2_(0.f,0.f); return exp(b*log(a)); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<float2_,	float2_>::strongest				pow(const float2_ a, const float2_ b)		{ if ((a.c.x==0.f)&&(a.c.y==0.f)) return float2_(0.f,0.f); return exp(b*log(a)); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<float2_,	double2_>::strongest			pow(const float2_ a, const double2_ b)		{ if ((a.c.x==0.f)&&(a.c.y==0.f)) return float2_(0.f,0.f); return exp(b*log(a)); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<double2_,	int2_>::strongest				pow(const double2_ a, const int2_ b)		{ if ((a.c.x==0.)&&(a.c.y==0.)) return double2_(0.,0.); return exp(b*log(a)); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<double2_,	float2_>::strongest				pow(const double2_ a, const float2_ b)		{ if ((a.c.x==0.f)&&(a.c.y==0.f)) return double2_(0.,0.); return exp(b*log(a)); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<double2_,	double2_>::strongest			pow(const double2_ a, const double2_ b)		{ if ((a.c.x==0.f)&&(a.c.y==0.f)) return double2_(0.,0.); return exp(b*log(a)); }

// --- Complex-Real
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<int2_,	float>::strongest				pow(const int2_ a, const float b)			{ float2_	c(b,0.f);			return pow(a,c); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<int2_,	double>::strongest				pow(const int2_ a, const double b)			{ double2_	c(b,0.);			return pow(a,c); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<float2_,	float>::strongest				pow(const float2_ a, const float b)			{ float2_	c(b,0.f);			return pow(a,c); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<float2_,	double>::strongest				pow(const float2_ a, const double b)		{ float2_	c(b,0.);			return pow(a,c); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<double2_,	float>::strongest				pow(const double2_ a, const float b)		{ double2_	c(b,0.);			return pow(a,c); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<double2_,	double>::strongest				pow(const double2_ a, const double b)		{ double2_	c(b,0.);			return pow(a,c); }

// --- Real-Complex
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<int,		int2_>::strongest				pow(const int a, const int2_ b)				{ float2_	c((float)a,0.f);	return pow(c,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<float,	int2_>::strongest				pow(const float a, const int2_ b)			{ float2_	c(a,0.f);			return pow(c,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<double,	int2_>::strongest				pow(const double a, const int2_ b)			{ double2_	c(a,0.);			return pow(c,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<int,		float2_>::strongest				pow(const int a, const float2_ b)			{ float2_	c((float)a,0.f);	return pow(c,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<float,	float2_>::strongest				pow(const float a, const float2_ b)			{ float2_	c(a,0.f);			return pow(c,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<double,	float2_>::strongest				pow(const double a, const float2_ b)		{ double2_	c(a,0.);			return pow(c,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<int,		double2_>::strongest			pow(const int a, const double2_ b)			{ float2_	c((float)a,0.f);	return pow(c,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<float,	double2_>::strongest			pow(const float a, const double2_ b)		{ float2_	c(a,0.);			return pow(c,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion_functions<double,	double2_>::strongest			pow(const double a, const double2_ b)		{ double2_	c(a,0.);			return pow(c,b); }

// --- Complex-Int (int2_)
__host__ __device__ __forceinline __forceinline__ float2_	pow(const float2_, const int);				
__host__ __device__ __forceinline __forceinline__ float2_	pow(const int2_ a, const int b)				
{ 
	switch (b)
	{
		case 0:		return int2_(1,0);
		case 1:		return a;
		case 2:		return a*a;
		case 3:		return a*a*a;
		case 4:		return a*a*a*a;
		case 5:		return a*a*a*a*a;
		case 6:		return a*a*a*a*a*a;
		case 7:		return a*a*a*a*a*a*a;
		case 8:		return a*a*a*a*a*a*a*a;
		case 9:		return a*a*a*a*a*a*a*a*a;
		case 10:	return a*a*a*a*a*a*a*a*a*a;
		case 11:	return a*a*a*a*a*a*a*a*a*a*a;
		case 12:	return a*a*a*a*a*a*a*a*a*a*a*a;
		default:
			if (b < 0 && b >= -12) return pow(1.f/a,-b);
			return pow(a, (float)b);
   }
}

// --- Complex-Int (float2_)
__host__ __device__ __forceinline __forceinline__ float2_	pow(const float2_ a, const int b)				
{ 
	switch (b)
	{
		case 0:		return float2_(1.f,0);
		case 1:		return a;
		case 2:		return a*a;
		case 3:		return a*a*a;
		case 4:		return a*a*a*a;
		case 5:		return a*a*a*a*a;
		case 6:		return a*a*a*a*a*a;
		case 7:		return a*a*a*a*a*a*a;
		case 8:		return a*a*a*a*a*a*a*a;
		case 9:		return a*a*a*a*a*a*a*a*a;
		case 10:	return a*a*a*a*a*a*a*a*a*a;
		case 11:	return a*a*a*a*a*a*a*a*a*a*a;
		case 12:	return a*a*a*a*a*a*a*a*a*a*a*a;
		default:
			if (b < 0 && b >= -12) return pow(1.f/a,-b);
			return pow(a, (float)b);
   }
}

// --- Complex-Int (double2_)
__host__ __device__ __forceinline __forceinline__ double2_	pow(const double2_ a, const int b)				
{ 
	switch (b)
	{
		case 0:		return double2_(1.f,0);
		case 1:		return a;
		case 2:		return a*a;
		case 3:		return a*a*a;
		case 4:		return a*a*a*a;
		case 5:		return a*a*a*a*a;
		case 6:		return a*a*a*a*a*a;
		case 7:		return a*a*a*a*a*a*a;
		case 8:		return a*a*a*a*a*a*a*a;
		case 9:		return a*a*a*a*a*a*a*a*a;
		case 10:	return a*a*a*a*a*a*a*a*a*a;
		case 11:	return a*a*a*a*a*a*a*a*a*a*a;
		case 12:	return a*a*a*a*a*a*a*a*a*a*a*a;
		default:
			if (b < 0 && b >= -12) return pow(1./a,-b);
			return pow(a, (double)b);
   }
}

/*********/
/* ASINH */  //OK
/*********/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions							asinh(const int a)							{ return asinh((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions							asinh(const int2_ a)						{ float2_	c = log(a + sqrt(a*a+1.0f));return float2_	(copysign(c.c.x, (float)a.c.x), copysign(c.c.y, (float)	a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions						asinh(const float2_ a)						{ float2_	c = log(a + sqrt(a*a+1.0f));	return float2_	(copysign(c.c.x,		a.c.x), copysign(c.c.y,			a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions						asinh(const double2_ a)						{ double2_	c = log(a + sqrt(a*a+1.0));	return double2_	(copysign(c.c.x,		a.c.x), copysign(c.c.y,			a.c.y)); }

/*********/
/* ACOSH */  //OK
/*********/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions							acosh(const int a)							{ return acosh((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions							acosh(const int2_ a)						{ float2_	c = log(a + sqrt(a*a-1.0f));	return float2_	(copysign(c.c.x, 0.f),	copysign(c.c.y, (float)	a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions						acosh(const float2_ a)						{ float2_	c = log(a + sqrt(a*a-1.0f));	return float2_	(copysign(c.c.x, 0.f),	copysign(c.c.y,			a.c.y)); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions						acosh(const double2_ a)						{ double2_	c = log(a + sqrt(a*a-1.0));	return double2_	(copysign(c.c.x, 0.),	copysign(c.c.y,			a.c.y)); }

/*********/
/* ATANH */  //OK
/*********/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions							atanh(const int a)							{ return atanh((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions							atanh(const int2_ a)						{ float2_	c = log((1.f+a)/(1.f-a))/2.f;	return c; }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions						atanh(const float2_ a)						{ float2_	c = log((1.f+a)/(1.f-a))/2.f;	return c; }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions						atanh(const double2_ a)						{ double2_	c = log((1.+a)/(1.-a))/2.;		return c; }

/********/
/* SINH */  //OK
/********/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions							sinh(const int a)							{ return sinh((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions							sinh(const int2_ a)							{ return sin(float2_(0.f,1.f)*a)/float2_(0.f,1.f); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions						sinh(const float2_ a)						{ return sin(float2_(0.f,1.f)*a)/float2_(0.f,1.f); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions						sinh(const double2_ a)						{ return sin(double2_(0.,1.)*a)/double2_(0.,1.); }

/********/
/* COSH */  //OK
/********/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions							cosh(const int a)							{ return cosh((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions							cosh(const int2_ a)							{ return cos(float2_(0.f,1.f)*a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions						cosh(const float2_ a)						{ return cos(float2_(0.f,1.f)*a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions						cosh(const double2_ a)						{ return cos(double2_(0.,1.)*a); }

/********/
/* TANH */  //OK
/********/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions							tanh(const int a)							{ return tanh((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions							tanh(const int2_ a)							{ return tan(float2_(0.f,1.f)*a)/float2_(0.f,1.f); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions						tanh(const float2_ a)						{ return tan(float2_(0.f,1.f)*a)/float2_(0.f,1.f); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions						tanh(const double2_ a)						{ return tan(double2_(0.,1.)*a)/double2_(0.f,1.f); }

/********/
/* ASIN */  //OK
/********/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions		asin(const int a)				{ return asin((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions		asin(const int2_ a)				{ return float2_(0.f,-1.f)*asinh(float2_(0.f,1.f)*a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions	asin(const float2_ a)			{ return float2_(0.f,-1.f)*asinh(float2_(0.f,1.f)*a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions	asin(const double2_ a)			{ return double2_(0.,-1.)*asinh(double2_(0.,1.)*a); }

/********/
/* ATAN */  //OK
/********/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions		atan(const int a)				{ return atan((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions		atan(const int2_ a)				{ return atanh(float2_(0.f,1.f)*a)/float2_(0.f,1.f); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions	atan(const float2_ a)			{ return atanh(float2_(0.f,1.f)*a)/float2_(0.f,1.f); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions	atan(const double2_ a)			{ return atanh(double2_(0.,1.)*a)/double2_(0.,1.); }

/********/
/* ACOS */  //OK
/********/
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int>::root_functions		acos(const int a)				{ return acos((float)a); }
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<int2_>::root_functions		acos(const int2_ a)				
					{	
 						if (a.c.x == 0) return float2_(pi_f/2., -(float)a.c.y); 
						return atan((sqrt(1-a*a))/a);
					}
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<float2_>::root_functions	acos(const float2_ a)			
					{	
 						if (a.c.x == 0.f) return float2_(pi_f/2., -a.c.y); 
						return atan((sqrt(1-a*a))/a);
					}
__host__ __device__ __forceinline __forceinline__ result_type_root_functions<double2_>::root_functions	acos(const double2_ a)	
					{	
 						if (a.c.x == 0.) return float2_(pi_d/2., -a.c.y); 
						return atan((sqrt(1-a*a))/a);
					}


'File written in /content/src/Function_Overloads.h'

In [18]:
%%cuda --name Operator_Overloads.h
/*********************************************/
/* AUXILIARY FUNCTION FOR operator+ OVERLOAD */
/*********************************************/
// --- Real-Real
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,int>::strongest				add(const int a,		const int b)		{ return a+b; };
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,float>::strongest			add(const int a,		const float b)		{ return a+b; };
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,int>::strongest			add(const float a,		const int b)		{ return a+b; };
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,double>::strongest			add(const int a,		const double b)		{ return a+b; };
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,int>::strongest			add(const double a,		const int b)		{ return a+b; };
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,float>::strongest			add(const float a,		const float b)		{ return a+b; };
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,double>::strongest		add(const float a,		const double b)		{ return a+b; };
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,float>::strongest		add(const double a,		const float b)		{ return a+b; };
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,double>::strongest		add(const double a,		const double b)		{ return a+b; };

// --- Complex-Real
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,int>::strongest			add(const int2_ a,		const int b)		{ result_type_promotion<int2_,int>::strongest				c; c.c.x = a.c.x+b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,float>::strongest			add(const int2_ a,		const float b)		{ result_type_promotion<int2_,float>::strongest				c; c.c.x = a.c.x+b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,double>::strongest		add(const int2_ a,		const double b)		{ result_type_promotion<int2_,double>::strongest			c; c.c.x = a.c.x+b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,int>::strongest			add(const float2_ a,	const int b)		{ result_type_promotion<float2_,int>::strongest				c; c.c.x = a.c.x+b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float>::strongest		add(const float2_ a,	const float b)		{ result_type_promotion<float2_,float>::strongest			c; c.c.x = a.c.x+b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,double>::strongest		add(const float2_ a,	const double b)		{ result_type_promotion<float2_,double>::strongest			c; c.c.x = a.c.x+b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,int>::strongest		add(const double2_ a,	const int b)		{ result_type_promotion<double2_,int>::strongest			c; c.c.x = a.c.x+b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,float>::strongest		add(const double2_ a,	const float b)		{ result_type_promotion<double2_,float>::strongest			c; c.c.x = a.c.x+b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,double>::strongest		add(const double2_ a,	const double b)		{ result_type_promotion<double2_,double>::strongest			c; c.c.x = a.c.x+b; c.c.y = a.c.y; return c; }

// --- Real-Complex
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,int2_>::strongest			add(const int a,		const int2_ b)		{ result_type_promotion<int,int2_>::strongest				c; c.c.x = a+b.c.x; c.c.y = b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,int2_>::strongest			add(const float a,		const int2_ b)		{ result_type_promotion<float,int2_>::strongest				c; c.c.x = a+b.c.x; c.c.y = b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,int2_>::strongest		add(const double a,		const int2_ b)		{ result_type_promotion<double,int2_>::strongest			c; c.c.x = a+b.c.x; c.c.y = b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,float2_>::strongest			add(const int a,		const float2_ b)	{ result_type_promotion<int,float2_>::strongest				c; c.c.x = a+b.c.x; c.c.y = b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,float2_>::strongest		add(const float a,		const float2_ b)	{ result_type_promotion<float,float2_>::strongest			c; c.c.x = a+b.c.x; c.c.y = b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,float2_>::strongest		add(const double a,		const float2_ b)	{ result_type_promotion<double,float2_>::strongest			c; c.c.x = a+b.c.x; c.c.y = b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,double2_>::strongest		add(const int a,		const double2_ b)	{ result_type_promotion<int,double2_>::strongest			c; c.c.x = a+b.c.x; c.c.y = b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,double2_>::strongest		add(const float a,		const double2_ b)	{ result_type_promotion<float,double2_>::strongest			c; c.c.x = a+b.c.x; c.c.y = b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,double2_>::strongest		add(const double a,		const double2_ b)	{ result_type_promotion<double,double2_>::strongest			c; c.c.x = a+b.c.x; c.c.y = b.c.y; return c; }

// --- Complex-Complex
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,int2_>::strongest			add(const int2_ a, const int2_ b)			{ result_type_promotion<int2_,int2_>::strongest				c; c.c.x = a.c.x+b.c.x; c.c.y = a.c.y+b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,float2_>::strongest		add(const int2_ a, const float2_ b)			{ result_type_promotion<int2_,float2_>::strongest			c; c.c.x = a.c.x+b.c.x; c.c.y = a.c.y+b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,int2_>::strongest		add(const float2_ a, const int2_ b)			{ result_type_promotion<float2_,int2_>::strongest			c; c.c.x = a.c.x+b.c.x; c.c.y = a.c.y+b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,double2_>::strongest		add(const int2_ a, const double2_ b)		{ result_type_promotion<int2_,double2_>::strongest			c; c.c.x = a.c.x+b.c.x; c.c.y = a.c.y+b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,int2_>::strongest		add(const double2_ a, const int2_ b)		{ result_type_promotion<double2_,int2_>::strongest			c; c.c.x = a.c.x+b.c.x; c.c.y = a.c.y+b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float2_>::strongest		add(const float2_ a, const float2_ b)		{ result_type_promotion<float2_,float2_>::strongest			c; c.c.x = a.c.x+b.c.x; c.c.y = a.c.y+b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,double2_>::strongest	add(const float2_ a, const double2_ b)		{ result_type_promotion<float2_,double2_>::strongest		c; c.c.x = a.c.x+b.c.x; c.c.y = a.c.y+b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,float2_>::strongest	add(const double2_ a, const float2_ b)		{ result_type_promotion<double2_,float2_>::strongest		c; c.c.x = a.c.x+b.c.x; c.c.y = a.c.y+b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,double2_>::strongest	add(const double2_ a, const double2_ b)		{ result_type_promotion<double2_,double2_>::strongest		c; c.c.x = a.c.x+b.c.x; c.c.y = a.c.y+b.c.y; return c; }

/*********************************************/
/* AUXILIARY FUNCTION FOR operator- OVERLOAD */
/*********************************************/
// --- Real-Real
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,int>::strongest			sub(const int a, const int b)				{ return a-b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,float>::strongest			sub(const int a, const float b)				{ return a-b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,int>::strongest			sub(const float a, const int b)				{ return a-b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,double>::strongest			sub(const int a, const double b)			{ return a-b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,int>::strongest			sub(const double a, const int b)			{ return a-b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,float>::strongest		sub(const float a, const float b)			{ return a-b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,double>::strongest		sub(const float a, const double b)			{ return a-b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,float>::strongest		sub(const double a, const float b)			{ return a-b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,double>::strongest		sub(const double a, const double b)			{ return a-b; }
// --- Complex-Real
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,int>::strongest			sub(const int2_ a, const int b)				{ result_type_promotion<int2_,int>::strongest				c; c.c.x = a.c.x-b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,float>::strongest		sub(const int2_ a, const float b)			{ result_type_promotion<int2_,float>::strongest				c; c.c.x = a.c.x-b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,double>::strongest		sub(const int2_ a, const double b)			{ result_type_promotion<int2_,double>::strongest			c; c.c.x = a.c.x-b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,int>::strongest		sub(const float2_ a, const int b)			{ result_type_promotion<float2_,int>::strongest				c; c.c.x = a.c.x-b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,float>::strongest		sub(const float2_ a, const float b)			{ result_type_promotion<float2_,float>::strongest			c; c.c.x = a.c.x-b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,double>::strongest		sub(const float2_ a, const double b)		{ result_type_promotion<float2_,double>::strongest			c; c.c.x = a.c.x-b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,int>::strongest		sub(const double2_ a, const int b)			{ result_type_promotion<double2_,int>::strongest			c; c.c.x = a.c.x-b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,float>::strongest		sub(const double2_ a, const float b)		{ result_type_promotion<double2_,float>::strongest			c; c.c.x = a.c.x-b; c.c.y = a.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,double>::strongest	sub(const double2_ a, const double b)		{ result_type_promotion<double2_,double>::strongest			c; c.c.x = a.c.x-b; c.c.y = a.c.y; return c; }
// --- Real-Complex
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,int2_>::strongest			sub(const int a, const int2_ b)				{ result_type_promotion<int,int2_>::strongest			c; c.c.x = a-b.c.x; c.c.y = -b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,int2_>::strongest		sub(const float a, const int2_ b)			{ result_type_promotion<float,int2_>::strongest			c; c.c.x = a-b.c.x; c.c.y = -b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,int2_>::strongest		sub(const double a, const int2_ b)			{ result_type_promotion<double,int2_>::strongest		c; c.c.x = a-b.c.x; c.c.y = -b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,float2_>::strongest		sub(const int a, const float2_ b)			{ result_type_promotion<int,float2_>::strongest			c; c.c.x = a-b.c.x; c.c.y = -b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,float2_>::strongest		sub(const float a, const float2_ b)			{ result_type_promotion<float,float2_>::strongest		c; c.c.x = a-b.c.x; c.c.y = -b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,float2_>::strongest		sub(const double a, const float2_ b)		{ result_type_promotion<double,float2_>::strongest		c; c.c.x = a-b.c.x; c.c.y = -b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,double2_>::strongest		sub(const int a, const double2_ b)			{ result_type_promotion<int,double2_>::strongest		c; c.c.x = a-b.c.x; c.c.y = -b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,double2_>::strongest		sub(const float a, const double2_ b)		{ result_type_promotion<float,double2_>::strongest		c; c.c.x = a-b.c.x; c.c.y = -b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,double2_>::strongest	sub(const double a, const double2_ b)		{ result_type_promotion<double,double2_>::strongest		c; c.c.x = a-b.c.x; c.c.y = -b.c.y; return c; }
// --- Complex-Complex
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,int2_>::strongest		sub(const int2_ a, const int2_ b)			{ result_type_promotion<int2_,int2_>::strongest			c; c.c.x = a.c.x-b.c.x; c.c.y = a.c.y-b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,float2_>::strongest		sub(const int2_ a, const float2_ b)			{ result_type_promotion<int2_,float2_>::strongest		c; c.c.x = a.c.x-b.c.x; c.c.y = a.c.y-b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,int2_>::strongest		sub(const float2_ a, const int2_ b)			{ result_type_promotion<float2_,int2_>::strongest		c; c.c.x = a.c.x-b.c.x; c.c.y = a.c.y-b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,double2_>::strongest		sub(const int2_ a, const double2_ b)		{ result_type_promotion<int2_,double2_>::strongest		c; c.c.x = a.c.x-b.c.x; c.c.y = a.c.y-b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,int2_>::strongest		sub(const double2_ a, const int2_ b)		{ result_type_promotion<double2_,int2_>::strongest		c; c.c.x = a.c.x-b.c.x; c.c.y = a.c.y-b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,float2_>::strongest	sub(const float2_ a, const float2_ b)		{ result_type_promotion<float2_,float2_>::strongest		c; c.c.x = a.c.x-b.c.x; c.c.y = a.c.y-b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,double2_>::strongest	sub(const float2_ a, const double2_ b)		{ result_type_promotion<float2_,double2_>::strongest	c; c.c.x = a.c.x-b.c.x; c.c.y = a.c.y-b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,float2_>::strongest	sub(const double2_ a, const float2_ b)		{ result_type_promotion<double2_,float2_>::strongest	c; c.c.x = a.c.x-b.c.x; c.c.y = a.c.y-b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,double2_>::strongest	sub(const double2_ a, const double2_ b)		{ result_type_promotion<double2_,double2_>::strongest	c; c.c.x = a.c.x-b.c.x; c.c.y = a.c.y-b.c.y; return c; }

/*********************************************/
/* AUXILIARY FUNCTION FOR operator* OVERLOAD */
/*********************************************/
// --- Real-Real
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,int>::strongest			mul(const int a, const int b)				{ return a*b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,float>::strongest			mul(const int a, const float b)				{ return a*b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,int>::strongest			mul(const float a, const int b)				{ return a*b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,double>::strongest			mul(const int a, const double b)			{ return a*b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,int>::strongest			mul(const double a, const int b)			{ return a*b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,float>::strongest		mul(const float a, const float b)			{ return a*b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,double>::strongest		mul(const float a, const double b)			{ return a*b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,float>::strongest		mul(const double a, const float b)			{ return a*b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,double>::strongest		mul(const double a, const double b)			{ return a*b; }
// --- Complex-Real
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,int>::strongest			mul(const int2_ a, const int b)				{ result_type_promotion<int2_,int>::strongest				c; c.c.x = a.c.x*b; c.c.y = a.c.y*b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,float>::strongest		mul(const int2_ a, const float b)			{ result_type_promotion<int2_,float>::strongest				c; c.c.x = a.c.x*b; c.c.y = a.c.y*b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,double>::strongest		mul(const int2_ a, const double b)			{ result_type_promotion<int2_,double>::strongest			c; c.c.x = a.c.x*b; c.c.y = a.c.y*b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,int>::strongest		mul(const float2_ a, const int b)			{ result_type_promotion<float2_,int>::strongest				c; c.c.x = a.c.x*b; c.c.y = a.c.y*b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,float>::strongest		mul(const float2_ a, const float b)			{ result_type_promotion<float2_,float>::strongest			c; c.c.x = a.c.x*b; c.c.y = a.c.y*b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,double>::strongest		mul(const float2_ a, const double b)		{ result_type_promotion<float2_,double>::strongest			c; c.c.x = a.c.x*b; c.c.y = a.c.y*b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,int>::strongest		mul(const double2_ a, const int b)			{ result_type_promotion<double2_,int>::strongest			c; c.c.x = a.c.x*b; c.c.y = a.c.y*b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,float>::strongest		mul(const double2_ a, const float b)		{ result_type_promotion<double2_,float>::strongest			c; c.c.x = a.c.x*b; c.c.y = a.c.y*b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,double>::strongest	mul(const double2_ a, const double b)		{ result_type_promotion<double2_,double>::strongest			c; c.c.x = a.c.x*b; c.c.y = a.c.y*b; return c; }
// --- Real-Complex
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,int2_>::strongest			mul(const int a, const int2_ b)				{ result_type_promotion<int,int2_>::strongest				c; c.c.x = a*b.c.x; c.c.y = a*b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,int2_>::strongest		mul(const float a, const int2_ b)			{ result_type_promotion<float,int2_>::strongest				c; c.c.x = a*b.c.x; c.c.y = a*b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,int2_>::strongest		mul(const double a, const int2_ b)			{ result_type_promotion<double,int2_>::strongest			c; c.c.x = a*b.c.x; c.c.y = a*b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,float2_>::strongest		mul(const int a, const float2_ b)			{ result_type_promotion<int,float2_>::strongest				c; c.c.x = a*b.c.x; c.c.y = a*b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,float2_>::strongest		mul(const float a, const float2_ b)			{ result_type_promotion<float,float2_>::strongest			c; c.c.x = a*b.c.x; c.c.y = a*b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,float2_>::strongest		mul(const double a, const float2_ b)		{ result_type_promotion<double,float2_>::strongest			c; c.c.x = a*b.c.x; c.c.y = a*b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,double2_>::strongest		mul(const int a, const double2_ b)			{ result_type_promotion<int,double2_>::strongest			c; c.c.x = a*b.c.x; c.c.y = a*b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,double2_>::strongest		mul(const float a, const double2_ b)		{ result_type_promotion<float,double2_>::strongest			c; c.c.x = a*b.c.x; c.c.y = a*b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,double2_>::strongest	mul(const double a, const double2_ b)		{ result_type_promotion<double,double2_>::strongest			c; c.c.x = a*b.c.x; c.c.y = a*b.c.y; return c; }
// --- Complex-Complex
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,int2_>::strongest			mul(const int2_ a, const int2_ b)			{ result_type_promotion<int2_,int2_>::strongest				c; c.c.x = a.c.x*b.c.x-a.c.y*b.c.y; c.c.y = a.c.y*b.c.x+a.c.x*b.c.y; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,float2_>::strongest		mul(const int2_ a, const float2_ b)			{ result_type_promotion<int2_,float2_>::strongest			c; c.c.x = a.c.x*b.c.x-a.c.y*b.c.y; c.c.y = a.c.y*b.c.x+a.c.x*b.c.y; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,int2_>::strongest		mul(const float2_ a, const int2_ b)			{ result_type_promotion<float2_,int2_>::strongest			c; c.c.x = a.c.x*b.c.x-a.c.y*b.c.y; c.c.y = a.c.y*b.c.x+a.c.x*b.c.y; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,double2_>::strongest		mul(const int2_ a, const double2_ b)		{ result_type_promotion<int2_,double2_>::strongest			c; c.c.x = a.c.x*b.c.x-a.c.y*b.c.y; c.c.y = a.c.y*b.c.x+a.c.x*b.c.y; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,int2_>::strongest		mul(const double2_ a, const int2_ b)		{ result_type_promotion<double2_,int2_>::strongest			c; c.c.x = a.c.x*b.c.x-a.c.y*b.c.y; c.c.y = a.c.y*b.c.x+a.c.x*b.c.y; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,float2_>::strongest		mul(const float2_ a, const float2_ b)		{ result_type_promotion<float2_,float2_>::strongest			c; c.c.x = a.c.x*b.c.x-a.c.y*b.c.y; c.c.y = a.c.y*b.c.x+a.c.x*b.c.y; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,double2_>::strongest		mul(const float2_ a, const double2_ b)		{ result_type_promotion<float2_,double2_>::strongest		c; c.c.x = a.c.x*b.c.x-a.c.y*b.c.y; c.c.y = a.c.y*b.c.x+a.c.x*b.c.y; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,float2_>::strongest		mul(const double2_ a, const float2_ b)		{ result_type_promotion<double2_,float2_>::strongest		c; c.c.x = a.c.x*b.c.x-a.c.y*b.c.y; c.c.y = a.c.y*b.c.x+a.c.x*b.c.y; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,double2_>::strongest	mul(const double2_ a, const double2_ b)		{ result_type_promotion<double2_,double2_>::strongest		c; c.c.x = a.c.x*b.c.x-a.c.y*b.c.y; c.c.y = a.c.y*b.c.x+a.c.x*b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,int2_>::strongest		mul(const int2_ a, const int2_ b)			{ result_type_promotion<int2_,int2_>::strongest				c; int k1, k2, k3; k1 = a.c.x*(b.c.x + b.c.y); k2 = b.c.y*(a.c.x + a.c.y); k3 = b.c.x*(a.c.y - a.c.x); c.c.x = k1 - k2; c.c.y = k1 + k3; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,float2_>::strongest		mul(const int2_ a, const float2_ b)			{ result_type_promotion<int2_,float2_>::strongest			c; float k1, k2, k3; k1 = a.c.x*(b.c.x + b.c.y); k2 = b.c.y*(a.c.x + a.c.y); k3 = b.c.x*(a.c.y - a.c.x); c.c.x = k1 - k2; c.c.y = k1 + k3; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,int2_>::strongest		mul(const float2_ a, const int2_ b)			{ result_type_promotion<float2_,int2_>::strongest			c; float k1, k2, k3; k1 = a.c.x*(b.c.x + b.c.y); k2 = b.c.y*(a.c.x + a.c.y); k3 = b.c.x*(a.c.y - a.c.x); c.c.x = k1 - k2; c.c.y = k1 + k3; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,double2_>::strongest		mul(const int2_ a, const double2_ b)		{ result_type_promotion<int2_,double2_>::strongest			c; double k1, k2, k3; k1 = a.c.x*(b.c.x + b.c.y); k2 = b.c.y*(a.c.x + a.c.y); k3 = b.c.x*(a.c.y - a.c.x); c.c.x = k1 - k2; c.c.y = k1 + k3; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,int2_>::strongest		mul(const double2_ a, const int2_ b)		{ result_type_promotion<double2_,int2_>::strongest			c; double k1, k2, k3; k1 = a.c.x*(b.c.x + b.c.y); k2 = b.c.y*(a.c.x + a.c.y); k3 = b.c.x*(a.c.y - a.c.x); c.c.x = k1 - k2; c.c.y = k1 + k3; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,float2_>::strongest	mul(const float2_ a, const float2_ b)		{ result_type_promotion<float2_,float2_>::strongest			c; float k1, k2, k3; k1 = a.c.x*(b.c.x + b.c.y); k2 = b.c.y*(a.c.x + a.c.y); k3 = b.c.x*(a.c.y - a.c.x); c.c.x = k1 - k2; c.c.y = k1 + k3; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,double2_>::strongest	mul(const float2_ a, const double2_ b)		{ result_type_promotion<float2_,double2_>::strongest		c; double k1, k2, k3; k1 = a.c.x*(b.c.x + b.c.y); k2 = b.c.y*(a.c.x + a.c.y); k3 = b.c.x*(a.c.y - a.c.x); c.c.x = k1 - k2; c.c.y = k1 + k3; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,float2_>::strongest	mul(const double2_ a, const float2_ b)		{ result_type_promotion<double2_,float2_>::strongest		c; double k1, k2, k3; k1 = a.c.x*(b.c.x + b.c.y); k2 = b.c.y*(a.c.x + a.c.y); k3 = b.c.x*(a.c.y - a.c.x); c.c.x = k1 - k2; c.c.y = k1 + k3; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,double2_>::strongest	mul(const double2_ a, const double2_ b)		{ result_type_promotion<double2_,double2_>::strongest		c; double k1, k2, k3; k1 = a.c.x*(b.c.x + b.c.y); k2 = b.c.y*(a.c.x + a.c.y); k3 = b.c.x*(a.c.y - a.c.x); c.c.x = k1 - k2; c.c.y = k1 + k3; return c; }

/*********************************************/
/* AUXILIARY FUNCTION FOR operator/ OVERLOAD */
/*********************************************/
// --- Real-Real
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,int>::strongest			div(const int a, const int b)				{ return a/b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,float>::strongest			div(const int a, const float b)				{ return a/b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,int>::strongest			div(const float a, const int b)				{ return a/b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,double>::strongest			div(const int a, const double b)			{ return a/b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,int>::strongest			div(const double a, const int b)			{ return a/b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,float>::strongest		div(const float a, const float b)			{ return a/b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,double>::strongest		div(const float a, const double b)			{ return a/b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,float>::strongest		div(const double a, const float b)			{ return a/b; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,double>::strongest		div(const double a, const double b)			{ return a/b; }
// --- Complex-Real
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,int>::strongest			div(const int2_ a, const int b)				{ result_type_promotion<int2_,int>::strongest				c; c.c.x = a.c.x/b; c.c.y = a.c.y/b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,float>::strongest		div(const int2_ a, const float b)			{ result_type_promotion<int2_,float>::strongest				c; c.c.x = a.c.x/b; c.c.y = a.c.y/b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,double>::strongest		div(const int2_ a, const double b)			{ result_type_promotion<int2_,double>::strongest			c; c.c.x = a.c.x/b; c.c.y = a.c.y/b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,int>::strongest		div(const float2_ a, const int b)			{ result_type_promotion<float2_,int>::strongest				c; c.c.x = a.c.x/b; c.c.y = a.c.y/b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,float>::strongest		div(const float2_ a, const float b)			{ result_type_promotion<float2_,float>::strongest			c; c.c.x = a.c.x/b; c.c.y = a.c.y/b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,double>::strongest		div(const float2_ a, const double b)		{ result_type_promotion<float2_,double>::strongest			c; c.c.x = a.c.x/b; c.c.y = a.c.y/b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,int>::strongest		div(const double2_ a, const int b)			{ result_type_promotion<double2_,int>::strongest			c; c.c.x = a.c.x/b; c.c.y = a.c.y/b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,float>::strongest		div(const double2_ a, const float b)		{ result_type_promotion<double2_,float>::strongest			c; c.c.x = a.c.x/b; c.c.y = a.c.y/b; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,double>::strongest	div(const double2_ a, const double b)		{ result_type_promotion<double2_,double>::strongest			c; c.c.x = a.c.x/b; c.c.y = a.c.y/b; return c; }
// --- Real-Complex
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,int2_>::strongest			div(const int a, const int2_ b)				{ result_type_promotion<int,int2_>::strongest				c; result_type_root<int,int2_>::root den;		den = b.c.x*b.c.x+b.c.y*b.c.y; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,int2_>::strongest		div(const float a, const int2_ b)			{ result_type_promotion<float,int2_>::strongest				c; result_type_root<float,int2_>::root den;		den = b.c.x*b.c.x+b.c.y*b.c.y; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,int2_>::strongest		div(const double a, const int2_ b)			{ result_type_promotion<double,int2_>::strongest			c; result_type_root<double,int2_>::root den;	den = b.c.x*b.c.x+b.c.y*b.c.y; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,float2_>::strongest		div(const int a, const float2_ b)			{ result_type_promotion<int,float2_>::strongest				c; result_type_root<int,float2_>::root den;		den = b.c.x*b.c.x+b.c.y*b.c.y; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,float2_>::strongest		div(const float a, const float2_ b)			{ result_type_promotion<float,float2_>::strongest			c; result_type_root<float,float2_>::root den;	den = b.c.x*b.c.x+b.c.y*b.c.y; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,float2_>::strongest		div(const double a, const float2_ b)		{ result_type_promotion<double,float2_>::strongest			c; result_type_root<double,float2_>::root den;	den = b.c.x*b.c.x+b.c.y*b.c.y; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,double2_>::strongest		div(const int a, const double2_ b)			{ result_type_promotion<int,double2_>::strongest			c; result_type_root<int,double2_>::root den;	den = b.c.x*b.c.x+b.c.y*b.c.y; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,double2_>::strongest		div(const float a, const double2_ b)		{ result_type_promotion<float,double2_>::strongest			c; result_type_root<float,double2_>::root den;	den = b.c.x*b.c.x+b.c.y*b.c.y; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,double2_>::strongest	div(const double a, const double2_ b)		{ result_type_promotion<double,double2_>::strongest			c; result_type_root<double,double2_>::root den; den = b.c.x*b.c.x+b.c.y*b.c.y; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,int2_>::strongest			div(const int a, const int2_ b)				{ result_type_promotion<int,int2_>::strongest				c; result_type_root<int,int2_>::root den;		den = hypot((float)b.c.x,(float)b.c.y); den = den*den; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,int2_>::strongest		div(const float a, const int2_ b)			{ result_type_promotion<float,int2_>::strongest				c; result_type_root<float,int2_>::root den;		den = hypot((float)b.c.x,(float)b.c.y); den = den*den; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,int2_>::strongest		div(const double a, const int2_ b)			{ result_type_promotion<double,int2_>::strongest			c; result_type_root<double,int2_>::root den;	den = hypot((float)b.c.x,(float)b.c.y); den = den*den; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,float2_>::strongest		div(const int a, const float2_ b)			{ result_type_promotion<int,float2_>::strongest				c; result_type_root<int,float2_>::root den;		den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,float2_>::strongest		div(const float a, const float2_ b)			{ result_type_promotion<float,float2_>::strongest			c; result_type_root<float,float2_>::root den;	den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,float2_>::strongest		div(const double a, const float2_ b)		{ result_type_promotion<double,float2_>::strongest			c; result_type_root<double,float2_>::root den;	den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int,double2_>::strongest		div(const int a, const double2_ b)			{ result_type_promotion<int,double2_>::strongest			c; result_type_root<int,double2_>::root den;	den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float,double2_>::strongest		div(const float a, const double2_ b)		{ result_type_promotion<float,double2_>::strongest			c; result_type_root<float,double2_>::root den;	den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double,double2_>::strongest	div(const double a, const double2_ b)		{ result_type_promotion<double,double2_>::strongest			c; result_type_root<double,double2_>::root den; den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = a*b.c.x/den; c.c.y = -a*b.c.y/den; return c; }
// --- Complex-Complex
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,int2_>::strongest		div(const int2_ a, const int2_ b)			{ result_type_promotion<int2_,int2_>::strongest				c; result_type_root<int2_,int2_>::root den;			den = (b.c.x*b.c.x)+(b.c.y*b.c.y); c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,float2_>::strongest		div(const int2_ a, const float2_ b)			{ result_type_promotion<int2_,float2_>::strongest			c; result_type_root<int2_,float2_>::root den;		den = (b.c.x*b.c.x)+(b.c.y*b.c.y); c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,int2_>::strongest		div(const float2_ a, const int2_ b)			{ result_type_promotion<float2_,int2_>::strongest			c; result_type_root<float2_,int2_>::root den;		den = (b.c.x*b.c.x)+(b.c.y*b.c.y); c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,double2_>::strongest		div(const int2_ a, const double2_ b)		{ result_type_promotion<int2_,double2_>::strongest			c; result_type_root<int2_,double2_>::root den;		den = (b.c.x*b.c.x)+(b.c.y*b.c.y); c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,int2_>::strongest		div(const double2_ a, const int2_ b)		{ result_type_promotion<double2_,int2_>::strongest			c; result_type_root<double2_,int2_>::root den;		den = (b.c.x*b.c.x)+(b.c.y*b.c.y); c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,float2_>::strongest	div(const float2_ a, const float2_ b)		{ result_type_promotion<float2_,float2_>::strongest			c; result_type_root<float2_,float2_>::root den;		den = (b.c.x*b.c.x)+(b.c.y*b.c.y); c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,double2_>::strongest	div(const float2_ a, const double2_ b)		{ result_type_promotion<float2_,double2_>::strongest		c; result_type_root<float2_,double2_>::root den;	den = (b.c.x*b.c.x)+(b.c.y*b.c.y); c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,float2_>::strongest	div(const double2_ a, const float2_ b)		{ result_type_promotion<double2_,float2_>::strongest		c; result_type_root<double2_,float2_>::root den;	den = (b.c.x*b.c.x)+(b.c.y*b.c.y); c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
//__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,double2_>::strongest	div(const double2_ a, const double2_ b)		{ result_type_promotion<double2_,double2_>::strongest		c; result_type_root<double2_,double2_>::root den;	den = (b.c.x*b.c.x)+(b.c.y*b.c.y); c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,int2_>::strongest		div(const int2_ a, const int2_ b)			{ result_type_promotion<int2_,int2_>::strongest				c; result_type_root<int2_,int2_>::root den;			den = hypot((float)b.c.x,(float)b.c.y); den = den*den; c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,float2_>::strongest		div(const int2_ a, const float2_ b)			{ result_type_promotion<int2_,float2_>::strongest			c; result_type_root<int2_,float2_>::root den;		den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,int2_>::strongest		div(const float2_ a, const int2_ b)			{ result_type_promotion<float2_,int2_>::strongest			c; result_type_root<float2_,int2_>::root den;		den = hypot((float)b.c.x,(float)b.c.y); den = den*den; c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<int2_,double2_>::strongest		div(const int2_ a, const double2_ b)		{ result_type_promotion<int2_,double2_>::strongest			c; result_type_root<int2_,double2_>::root den;		den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,int2_>::strongest		div(const double2_ a, const int2_ b)		{ result_type_promotion<double2_,int2_>::strongest			c; result_type_root<double2_,int2_>::root den;		den = hypot((float)b.c.x,(float)b.c.y); den = den*den; c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,float2_>::strongest	div(const float2_ a, const float2_ b)		{ result_type_promotion<float2_,float2_>::strongest			c; result_type_root<float2_,float2_>::root den;		den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<float2_,double2_>::strongest	div(const float2_ a, const double2_ b)		{ result_type_promotion<float2_,double2_>::strongest		c; result_type_root<float2_,double2_>::root den;	den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,float2_>::strongest	div(const double2_ a, const float2_ b)		{ result_type_promotion<double2_,float2_>::strongest		c; result_type_root<double2_,float2_>::root den;	den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }
__host__ __device__ __forceinline __forceinline__  result_type_promotion<double2_,double2_>::strongest	div(const double2_ a, const double2_ b)		{ result_type_promotion<double2_,double2_>::strongest		c; result_type_root<double2_,double2_>::root den;	den = hypot(b.c.x,b.c.y); den = den*den; c.c.x = ((a.c.x*b.c.x)+(a.c.y*b.c.y))/den; c.c.y = ((a.c.y*b.c.x)-(a.c.x*b.c.y))/den; return c; }

/*******************************/
/* INSTANTIATIONS OF operator+ */
/*******************************/
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,int2_>::strongest operator+(const int a,const int2_ b)							{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,float2_>::strongest operator+(const int a,const float2_ b)						{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,double2_>::strongest operator+(const int a,const double2_ b)					{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,int2_>::strongest operator+(const float a,const int2_ b)						{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,float2_>::strongest operator+(const float a,const float2_ b)					{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,double2_>::strongest operator+(const float a,const double2_ b)				{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,int2_>::strongest operator+(const double a,const int2_ b)					{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,float2_>::strongest operator+(const double a,const float2_ b)				{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,double2_>::strongest operator+(const double a,const double2_ b)				{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,int>::strongest operator+(const int2_ a,const int b)							{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,float>::strongest operator+(const int2_ a,const float b)						{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,double>::strongest operator+(const int2_ a,const double b)					{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,int2_>::strongest operator+(const int2_ a,const int2_ b)						{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,float2_>::strongest operator+(const int2_ a,const float2_ b)					{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,double2_>::strongest operator+(const int2_ a,const double2_ b)				{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,int>::strongest operator+(const float2_ a,const int b)						{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float>::strongest operator+(const float2_ a,const float b)					{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,double>::strongest operator+(const float2_ a,const double b)				{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,int2_>::strongest operator+(const float2_ a,const int2_ b)					{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float2_>::strongest operator+(const float2_ a,const float2_ b)				{ result_type_promotion<float2_,float2_>::strongest			c; c.c.x = a.c.x+b.c.x; c.c.y = a.c.y+b.c.y; return c; }

//__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float2_>::strongest operator+(const float2_ a,const float2_ b)				{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,double2_>::strongest operator+(const float2_ a,const double2_ b)			{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,int>::strongest operator+(const double2_ a,const int b)					{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,float>::strongest operator+(const double2_ a,const float b)				{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,double>::strongest operator+(const double2_ a,const double b)				{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,int2_>::strongest operator+(const double2_ a,const int2_ b)				{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,float2_>::strongest operator+(const double2_ a,const float2_ b)			{ return add(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,double2_>::strongest operator+(const double2_ a,const double2_ b)			{ return add(a,b); }

/*******************************/
/* INSTANTIATIONS OF operator- */
/*******************************/
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,int2_>::strongest operator-(const int a,const int2_ b)							{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,float2_>::strongest operator-(const int a,const float2_ b)						{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,double2_>::strongest operator-(const int a,const double2_ b)					{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,int2_>::strongest operator-(const float a,const int2_ b)						{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,float2_>::strongest operator-(const float a,const float2_ b)					{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,double2_>::strongest operator-(const float a,const double2_ b)				{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,int2_>::strongest operator-(const double a,const int2_ b)					{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,float2_>::strongest operator-(const double a,const float2_ b)				{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,double2_>::strongest operator-(const double a,const double2_ b)				{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,int>::strongest operator-(const int2_ a,const int b)							{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,float>::strongest operator-(const int2_ a,const float b)						{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,double>::strongest operator-(const int2_ a,const double b)					{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,int2_>::strongest operator-(const int2_ a,const int2_ b)						{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,float2_>::strongest operator-(const int2_ a,const float2_ b)					{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,double2_>::strongest operator-(const int2_ a,const double2_ b)				{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,int>::strongest operator-(const float2_ a,const int b)						{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float>::strongest operator-(const float2_ a,const float b)					{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,double>::strongest operator-(const float2_ a,const double b)				{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,int2_>::strongest operator-(const float2_ a,const int2_ b)					{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float2_>::strongest operator-(const float2_ a,const float2_ b)				{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,double2_>::strongest operator-(const float2_ a,const double2_ b)			{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,int>::strongest operator-(const double2_ a,const int b)					{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,float>::strongest operator-(const double2_ a,const float b)				{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,double>::strongest operator-(const double2_ a,const double b)				{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,int2_>::strongest operator-(const double2_ a,const int2_ b)				{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,float2_>::strongest operator-(const double2_ a,const float2_ b)			{ return sub(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,double2_>::strongest operator-(const double2_ a,const double2_ b)			{ return sub(a,b); }

/*******************************/
/* INSTANTIATIONS OF operator* */
/*******************************/
// http://www.embedded.com/design/embedded/4007256/Digital-Signal-Processing-Tricks--Fast-multiplication-of-complex-numbers%5D

__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,int2_>::strongest operator*(const int a,const int2_ b)							{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,float2_>::strongest operator*(const int a,const float2_ b)						{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,double2_>::strongest operator*(const int a,const double2_ b)					{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,int2_>::strongest operator*(const float a,const int2_ b)						{ return mul(a,b); }
//__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,float2_>::strongest operator*(const float a,const float2_ b)					{ result_type_promotion<float,float2_>::strongest			c; c.c.x = a*b.c.x; c.c.y = a*b.c.y; return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,float2_>::strongest operator*(const float a,const float2_ b)					{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,double2_>::strongest operator*(const float a,const double2_ b)				{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,int2_>::strongest operator*(const double a,const int2_ b)					{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,float2_>::strongest operator*(const double a,const float2_ b)				{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,double2_>::strongest operator*(const double a,const double2_ b)				{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,int>::strongest operator*(const int2_ a,const int b)							{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,float>::strongest operator*(const int2_ a,const float b)						{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,double>::strongest operator*(const int2_ a,const double b)					{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,int2_>::strongest operator*(const int2_ a,const int2_ b)						{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,float2_>::strongest operator*(const int2_ a,const float2_ b)					{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,double2_>::strongest operator*(const int2_ a,const double2_ b)				{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,int>::strongest operator*(const float2_ a,const int b)						{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float>::strongest operator*(const float2_ a,const float b)					{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,double>::strongest operator*(const float2_ a,const double b)				{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,int2_>::strongest operator*(const float2_ a,const int2_ b)					{ return mul(a,b); }
 __host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float2_>::strongest operator*(const float2_ a,const float2_ b)				{ return mul(a,b); }
//__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float2_>::strongest operator*(const float2_ a,const float2_ b)				{ result_type_promotion<float2_,float2_>::strongest			c; c.c.x = a.c.x*b.c.x-a.c.y*b.c.y; c.c.y=a.c.x*b.c.y+a.c.y*b.c.x; printf("%f %f %f %f\n",c.c.x,c.c.y,a.c.x,a.c.y); return c; }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,double2_>::strongest operator*(const float2_ a,const double2_ b)			{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,int>::strongest operator*(const double2_ a,const int b)					{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,float>::strongest operator*(const double2_ a,const float b)				{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,double>::strongest operator*(const double2_ a,const double b)				{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,int2_>::strongest operator*(const double2_ a,const int2_ b)				{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,float2_>::strongest operator*(const double2_ a,const float2_ b)			{ return mul(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,double2_>::strongest operator*(const double2_ a,const double2_ b)			{ return mul(a,b); }



/*******************************/
/* INSTANTIATIONS OF operator/ */
/*******************************/
// http://www.embedded.com/design/embedded/4007256/Digital-Signal-Processing-Tricks--Fast-multiplication-of-complex-numbers%5D

__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,int2_>::strongest operator/(const int a,const int2_ b)							{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,float2_>::strongest operator/(const int a,const float2_ b)						{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int,double2_>::strongest operator/(const int a,const double2_ b)					{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,int2_>::strongest operator/(const float a,const int2_ b)						{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,float2_>::strongest operator/(const float a,const float2_ b)					{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float,double2_>::strongest operator/(const float a,const double2_ b)				{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,int2_>::strongest operator/(const double a,const int2_ b)					{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,float2_>::strongest operator/(const double a,const float2_ b)				{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double,double2_>::strongest operator/(const double a,const double2_ b)				{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,int>::strongest operator/(const int2_ a,const int b)							{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,float>::strongest operator/(const int2_ a,const float b)						{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,double>::strongest operator/(const int2_ a,const double b)					{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,int2_>::strongest operator/(const int2_ a,const int2_ b)						{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,float2_>::strongest operator/(const int2_ a,const float2_ b)					{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<int2_,double2_>::strongest operator/(const int2_ a,const double2_ b)				{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,int>::strongest operator/(const float2_ a,const int b)						{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float>::strongest operator/(const float2_ a,const float b)					{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,double>::strongest operator/(const float2_ a,const double b)				{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,int2_>::strongest operator/(const float2_ a,const int2_ b)					{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,float2_>::strongest operator/(const float2_ a,const float2_ b)				{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<float2_,double2_>::strongest operator/(const float2_ a,const double2_ b)			{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,int>::strongest operator/(const double2_ a,const int b)					{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,float>::strongest operator/(const double2_ a,const float b)				{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,double>::strongest operator/(const double2_ a,const double b)				{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,int2_>::strongest operator/(const double2_ a,const int2_ b)				{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,float2_>::strongest operator/(const double2_ a,const float2_ b)			{ return div(a,b); }
__host__ __device__ __forceinline __forceinline__ result_type_promotion<double2_,double2_>::strongest operator/(const double2_ a,const double2_ b)			{ return div(a,b); }


'File written in /content/src/Operator_Overloads.h'

In [19]:
%%cuda --name Result_Types.h
#ifndef __RESULT_TYPES_CUH__
#define __RESULT_TYPES_CUH__

/*************/
/* PROMOTION */
/*************/

template <typename, typename> struct result_type_promotion;

// int-int, float-float, double-double, int2_-int2_, float2_-float2_-, double2_-double2_
template<> struct result_type_promotion< int,		int >			{ typedef int strongest; };
template<> struct result_type_promotion< float,		float >			{ typedef float strongest; };
template<> struct result_type_promotion< double,	double >		{ typedef double strongest; };
template<> struct result_type_promotion< int2_,		int2_ >			{ typedef int2_ strongest; };
template<> struct result_type_promotion< float2_,	float2_ >		{ typedef float2_ strongest; };
template<> struct result_type_promotion< double2_,	double2_ >		{ typedef double2_ strongest; };

// --- From int to others
template<> struct result_type_promotion< float,		int >			{ typedef float strongest; };
template<> struct result_type_promotion< int,		float >			{ typedef float strongest; };

template<> struct result_type_promotion< double,	int >			{ typedef double strongest; };
template<> struct result_type_promotion< int,		double >		{ typedef double strongest; };

template<> struct result_type_promotion< int2_,		int >			{ typedef int2_ strongest; };
template<> struct result_type_promotion< int,		int2_ >			{ typedef int2_ strongest; };

template<> struct result_type_promotion< float2_,	int >			{ typedef float2_ strongest; };
template<> struct result_type_promotion< int,		float2_ >		{ typedef float2_ strongest; };

template<> struct result_type_promotion< double2_,	int >			{ typedef double2_ strongest; };
template<> struct result_type_promotion< int,		double2_ >		{ typedef double2_ strongest; };

// --- From float to others
template<> struct result_type_promotion< double,	float >			{ typedef double strongest; };
template<> struct result_type_promotion< float,		double >		{ typedef double strongest; };

template<> struct result_type_promotion< int2_,		float >			{ typedef float2_ strongest; };
template<> struct result_type_promotion< float,		int2_ >			{ typedef float2_ strongest; };

template<> struct result_type_promotion< float2_,	float >			{ typedef float2_ strongest; };
template<> struct result_type_promotion< float,		float2_ >		{ typedef float2_ strongest; };

template<> struct result_type_promotion< double2_,	float >			{ typedef double2_ strongest; };
template<> struct result_type_promotion< float,		double2_ >		{ typedef double2_ strongest; };

// --- From double to others
template<> struct result_type_promotion< int2_,		double >		{ typedef double2_ strongest; };
template<> struct result_type_promotion< double,	int2_ >			{ typedef double2_ strongest; };

template<> struct result_type_promotion< float2_,	double >		{ typedef double2_ strongest; };
template<> struct result_type_promotion< double,	float2_ >		{ typedef double2_ strongest; };

template<> struct result_type_promotion< double2_,	double >		{ typedef double2_ strongest; };
template<> struct result_type_promotion< double,	double2_ >		{ typedef double2_ strongest; };

// --- From int2_ to others
template<> struct result_type_promotion< float2_,	int2_ >			{ typedef float2_ strongest; };
template<> struct result_type_promotion< int2_,		float2_ >		{ typedef float2_ strongest; };

template<> struct result_type_promotion< double2_,	int2_ >			{ typedef double2_ strongest; };
template<> struct result_type_promotion< int2_,		double2_ >		{ typedef double2_ strongest; };

// --- From float2_ to others
template<> struct result_type_promotion< double2_,	float2_ >		{ typedef double2_ strongest; };
template<> struct result_type_promotion< float2_,	double2_ >		{ typedef double2_ strongest; };

/***********************/
/* PROMOTION FUNCTIONS */
/***********************/

template <typename, typename> struct result_type_promotion_functions;

// int-int, float-float, double-double, int2_-int2_, float2_-float2_-, double2_-double2_
template<> struct result_type_promotion_functions< int,		int >			{ typedef float strongest; };
template<> struct result_type_promotion_functions< float,	float >			{ typedef float strongest; };
template<> struct result_type_promotion_functions< double,	double >		{ typedef double strongest; };
template<> struct result_type_promotion_functions< int2_,	int2_ >			{ typedef float2_ strongest; };
template<> struct result_type_promotion_functions< float2_,	float2_ >		{ typedef float2_ strongest; };
template<> struct result_type_promotion_functions< double2_,double2_ >		{ typedef double2_ strongest; };

// --- From int to others
template<> struct result_type_promotion_functions< float,	int >			{ typedef float strongest; };
template<> struct result_type_promotion_functions< int,		float >			{ typedef float strongest; };

template<> struct result_type_promotion_functions< double,	int >			{ typedef double strongest; };
template<> struct result_type_promotion_functions< int,		double >		{ typedef double strongest; };

template<> struct result_type_promotion_functions< int2_,	int >			{ typedef float2_ strongest; };
template<> struct result_type_promotion_functions< int,		int2_ >			{ typedef float2_ strongest; };

template<> struct result_type_promotion_functions< float2_,	int >			{ typedef float2_ strongest; };
template<> struct result_type_promotion_functions< int,		float2_ >		{ typedef float2_ strongest; };

template<> struct result_type_promotion_functions< double2_,int >			{ typedef double2_ strongest; };
template<> struct result_type_promotion_functions< int,		double2_ >		{ typedef double2_ strongest; };

// --- From float to others
template<> struct result_type_promotion_functions< double,	float >			{ typedef double strongest; };
template<> struct result_type_promotion_functions< float,	double >		{ typedef double strongest; };

template<> struct result_type_promotion_functions< int2_,	float >			{ typedef float2_ strongest; };
template<> struct result_type_promotion_functions< float,	int2_ >			{ typedef float2_ strongest; };

template<> struct result_type_promotion_functions< float2_,	float >			{ typedef float2_ strongest; };
template<> struct result_type_promotion_functions< float,	float2_ >		{ typedef float2_ strongest; };

template<> struct result_type_promotion_functions< double2_,float >			{ typedef double2_ strongest; };
template<> struct result_type_promotion_functions< float,	double2_ >		{ typedef double2_ strongest; };

// --- From double to others
template<> struct result_type_promotion_functions< int2_,	double >		{ typedef double2_ strongest; };
template<> struct result_type_promotion_functions< double,	int2_ >			{ typedef double2_ strongest; };

template<> struct result_type_promotion_functions< float2_,	double >		{ typedef double2_ strongest; };
template<> struct result_type_promotion_functions< double,	float2_ >		{ typedef double2_ strongest; };

template<> struct result_type_promotion_functions< double2_,double >		{ typedef double2_ strongest; };
template<> struct result_type_promotion_functions< double,	double2_ >		{ typedef double2_ strongest; };

// --- From int2_ to others
template<> struct result_type_promotion_functions< float2_,	int2_ >			{ typedef float2_ strongest; };
template<> struct result_type_promotion_functions< int2_,	float2_ >		{ typedef float2_ strongest; };

template<> struct result_type_promotion_functions< double2_,int2_ >			{ typedef double2_ strongest; };
template<> struct result_type_promotion_functions< int2_,	double2_ >		{ typedef double2_ strongest; };

// --- From float2_ to others
template<> struct result_type_promotion_functions< double2_,float2_ >		{ typedef double2_ strongest; };
template<> struct result_type_promotion_functions< float2_,	double2_ >		{ typedef double2_ strongest; };

/********/
/* ROOT */
/********/

template <typename, typename> struct result_type_root;

// int-int, float-float, double-double
template<> struct result_type_root< int,		int >			{ typedef int strongest; };
template<> struct result_type_root< float,		float >			{ typedef float strongest; };
template<> struct result_type_root< double,		double >		{ typedef double strongest; };

// --- From int to others
template<> struct result_type_root< float,		int >			{ typedef float root; };
template<> struct result_type_root< int,		float >			{ typedef float root; };

template<> struct result_type_root< double,		int >			{ typedef double root; };
template<> struct result_type_root< int,		double >		{ typedef double root; };

template<> struct result_type_root< int2_,		int >			{ typedef int	root; };
template<> struct result_type_root< int,		int2_ >			{ typedef int	root; };

template<> struct result_type_root< float2_,	int >			{ typedef float root; };
template<> struct result_type_root< int,		float2_ >		{ typedef float root; };

template<> struct result_type_root< double2_,	int >			{ typedef double root; };
template<> struct result_type_root< int,		double2_ >		{ typedef double root; };

// --- From float to others
template<> struct result_type_root< double,		float >			{ typedef double root; };
template<> struct result_type_root< float,		double >		{ typedef double root; };

template<> struct result_type_root< int2_,		float >			{ typedef float root; };
template<> struct result_type_root< float,		int2_ >			{ typedef float root; };

template<> struct result_type_root< float2_,	float >			{ typedef float root; };
template<> struct result_type_root< float,		float2_ >		{ typedef float root; };

template<> struct result_type_root< double2_,	float >			{ typedef double root; };
template<> struct result_type_root< float,		double2_ >		{ typedef double root; };

// --- From double to others
template<> struct result_type_root< int2_,		double >		{ typedef double root; };
template<> struct result_type_root< double,		int2_ >			{ typedef double root; };

template<> struct result_type_root< float2_,	double >		{ typedef double root; };
template<> struct result_type_root< double,		float2_ >		{ typedef double root; };

template<> struct result_type_root< double2_,	double >		{ typedef double root; };
template<> struct result_type_root< double,		double2_ >		{ typedef double root; };

// --- From int2_ to others
template<> struct result_type_root< int2_,		int2_ >			{ typedef int root; };

template<> struct result_type_root< float2_,	int2_ >			{ typedef float root; };
template<> struct result_type_root< int2_,		float2_ >		{ typedef float root; };

template<> struct result_type_root< double2_,	int2_ >			{ typedef double root; };
template<> struct result_type_root< int2_,		double2_ >		{ typedef double root; };

// --- From float2_ to others
template<> struct result_type_root< float2_,	float2_ >		{ typedef float root; };

template<> struct result_type_root< double2_,	float2_ >		{ typedef double root; };
template<> struct result_type_root< float2_,	double2_ >		{ typedef double root; };
	
// --- From double2_ to others
template<> struct result_type_root< double2_,	double2_ >		{ typedef double root; };


/****************/
/* ROOT REDUCED */
/****************/

template <typename> struct result_type_root_reduced;

template<> struct result_type_root_reduced<int>			{ typedef int root_reduced; };
template<> struct result_type_root_reduced<float>		{ typedef float root_reduced; };
template<> struct result_type_root_reduced<double>		{ typedef double root_reduced; };
template<> struct result_type_root_reduced<int2_>		{ typedef int root_reduced; };
template<> struct result_type_root_reduced<float2_>		{ typedef float root_reduced; };
template<> struct result_type_root_reduced<double2_>	{ typedef double root_reduced; };

/******************/
/* ROOT AUGMENTED */
/******************/

template <typename> struct result_type_root_augmented;

template<> struct result_type_root_augmented<int>		{ typedef float2_ root_augmented; };
template<> struct result_type_root_augmented<float>		{ typedef float2_ root_augmented; };
template<> struct result_type_root_augmented<double>	{ typedef double2_ root_augmented; };
template<> struct result_type_root_augmented<int2_>		{ typedef int2_ root_augmented; };
template<> struct result_type_root_augmented<float2_>	{ typedef float2_ root_augmented; };
template<> struct result_type_root_augmented<double2_>	{ typedef double2_ root_augmented; };

/*************/
/* FUNCTIONS */
/*************/

template <typename> struct result_type_root_functions;

template<> struct result_type_root_functions<int>		{ typedef float root_functions; };
template<> struct result_type_root_functions<float>		{ typedef float root_functions; };
template<> struct result_type_root_functions<double>	{ typedef double root_functions; };
template<> struct result_type_root_functions<int2_>		{ typedef float2_ root_functions; };
template<> struct result_type_root_functions<float2_>	{ typedef float2_ root_functions; };
template<> struct result_type_root_functions<double2_>	{ typedef double2_ root_functions; };

/*********************/
/* FUNCTIONS REDUCED */
/*********************/

template <typename> struct result_type_root_functions_reduced;

template<> struct result_type_root_functions_reduced<int>		{ typedef float root_functions_reduced; };
template<> struct result_type_root_functions_reduced<float>		{ typedef float root_functions_reduced; };
template<> struct result_type_root_functions_reduced<double>	{ typedef double root_functions_reduced; };
template<> struct result_type_root_functions_reduced<int2_>		{ typedef float root_functions_reduced; };
template<> struct result_type_root_functions_reduced<float2_>	{ typedef float root_functions_reduced; };
template<> struct result_type_root_functions_reduced<double2_>	{ typedef double root_functions_reduced; };

#endif

'File written in /content/src/Result_Types.h'

Compilazione

In [25]:
!nvcc -rdc=true -o "/content/src/kernel.o" /content/src/kernel.cu /content/src/InputOutput.cu /content/src/TimingCPU.cu /content/src/TimingGPU.cu /content/src/Utilities.cu -lcusolver -lcublas

/content/src/kernel.cu(44): warning: variable "h_U" was declared but never referenced

/content/src/kernel.cu(45): warning: variable "h_V" was declared but never referenced

/content/src/kernel.cu(76): warning: variable "econ" was declared but never referenced

/content/src/kernel.cu(79): warning: variable "residual" was declared but never referenced

/content/src/kernel.cu(80): warning: variable "executedSweeps" was declared but never referenced



In [26]:
!chmod 755 /content/src/kernel.o

In [27]:
!/content/src/kernel.o

Calculation of the singular values only: 130.884293 ms

gesvdj converges 
